In [1]:
import pandas as pd
import numpy as np
import networkx as nx
from pyhive import hive
import pydoop.hdfs as hdfs
from pyhive import hive
from impala.dbapi import connect
from impala.util import as_pandas  

In [2]:
# Establishing the connection with Hive
host_name = "10.107.6.14"
port = 10000
user = "hdfs"
password = "bigid#01"
database="referral_mart_latest"
conn = hive.Connection(host=host_name, port=port, username=user, password=password,
                           database=database, auth='CUSTOM')
cursor_1 = conn.cursor()

**Create splits for customer group id and create partition of service data and other tables to make it computationally less intensive. If enough processing capacity is available or data is already partitioned skip this step**

In [3]:
cust_grp = pd.read_csv('Cust_grp_master_final_1.csv')

/usr/local/src/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0,1,2,3,5,6,7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
print(cust_grp.shape)
print(cust_grp.columns)

(1924867, 10)
Index(['con_id', 'con_id_x', 'con_id_y', 'phone_no', 'repm_vin', 'ser_ph_grp',
       'service_phone', 'unique_cust_grp', 'vin_grp', 'Target'],
      dtype='object')


In [5]:
cust_grp['unique_cust_grp'].value_counts()

165.0         8206
30_used2      3104
13.0          2049
3808.0         938
158_used2      464
6633.0         439
52394.0        400
841_used2      354
60474.0        337
11187.0        300
21466.0        275
165.0          251
2994.0         238
4901.0         218
448_used2      207
106517.0       202
64071.0        197
1290_used2     190
2971_used2     184
75633.0        180
1294_used2     179
575_used2      176
10474.0        175
41082.0        157
39769.0        148
34686.0        147
31066.0        143
29691.0        137
66352.0        136
12479.0        134
              ... 
825081.0         1
825082.0         1
825083.0         1
825084.0         1
825088.0         1
825057.0         1
825090.0         1
825092.0         1
825093.0         1
825094.0         1
825096.0         1
825098.0         1
825078.0         1
825077.0         1
825076.0         1
825075.0         1
825073.0         1
825072.0         1
825071.0         1
825070.0         1
825068.0         1
825067.0    

In [4]:
cust_grp.head(3)

,con_id,con_id_x,con_id_y,phone_no,repm_vin,ser_ph_grp,service_phone,unique_cust_grp,vin_grp,Target
0,NaN,1-29OZ-1503,1-29OZ-1503,9.89111e+09,MALAA51HR5M700082,1316486,9891112121,0,566852,0
1,NaN,1-29OE-2971,1-29OE-2971,2.5012e+09,MALAA51HR5M700158,235,2501201900,1,566853,0
2,NaN,1-2A9X-3345,1-2A9X-3345,9.99416e+09,MALAA51HR5M700178,1567815,9994155040,2,566854,0


In [7]:
#Separate customer group containing used vehicles where used_1 is the first owner of used vehicle
#and used_2 is the 2nd owner of a used vehicle
cust_grp['used_1'] = np.where(((cust_grp['vin_grp']=='used_car_1st')|
                             (cust_grp['unique_cust_grp'].astype(str).str.contains('_used'))), True, False)
cust_grp['used_2'] = np.where(((cust_grp['vin_grp']=='used_car_2nd')|
                             (cust_grp['unique_cust_grp'].astype(str).str.contains('_used2'))), True, False)

In [8]:
used_cust_grp_1 = cust_grp[cust_grp['used_1']==True][['repm_vin', 'service_phone', 'unique_cust_grp', 'con_id']]
used_cust_grp_2 = cust_grp[cust_grp['used_2']==True][['repm_vin', 'service_phone', 'unique_cust_grp', 'con_id']]
print("used_cust_grp_1 : ",used_cust_grp_1.shape)
print("used_cust_grp_2 : ",used_cust_grp_2.shape)

used_cust_grp_1 :  (232003, 4)
used_cust_grp_2 :  (187221, 4)


In [9]:
print(cust_grp.shape)
print(cust_grp.columns)

(1924867, 12)
Index(['con_id', 'con_id_x', 'con_id_y', 'phone_no', 'repm_vin', 'ser_ph_grp',
       'service_phone', 'unique_cust_grp', 'vin_grp', 'Target', 'used_1',
       'used_2'],
      dtype='object')


In [49]:
cust_grp.head(3)

,con_id,con_id_x,con_id_y,phone_no,repm_vin,ser_ph_grp,service_phone,unique_cust_grp,vin_grp,Target,used_1,used_2,used_vin
0,NaN,1-29OZ-1503,1-29OZ-1503,9.89111e+09,MALAA51HR5M700082,1316486,9891112121,0,566852,0,False,False,False
1,NaN,1-29OE-2971,1-29OE-2971,2.5012e+09,MALAA51HR5M700158,235,2501201900,1,566853,0,False,False,False
2,NaN,1-2A9X-3345,1-2A9X-3345,9.99416e+09,MALAA51HR5M700178,1567815,9994155040,2,566854,0,False,False,False


In [11]:
# Creating the feature used_vin such that used_vin == True means its a used vehicle and used_vin == False means its a new car
cust_grp['used_vin'] = np.where(cust_grp['used_1'] == True, True, False)
cust_grp['used_vin'] = np.where(cust_grp['used_2'] == True, True, False)

In [12]:
cust_grp['used_vin'].value_counts()

False    1737646
True      187221
Name: used_vin, dtype: int64

In [13]:
import glob
#Create df of customer group containing non-used vehicles only 
used_vin_list = cust_grp[(cust_grp['used_vin'] == True)].repm_vin.unique().tolist()
len(used_vin_list)

127379

In [14]:
cust_grp_noused = cust_grp[~cust_grp.unique_cust_grp.isin(used_vin_list)]
cust_grp_noused.shape

(1924867, 13)

In [15]:
cust_grp_noused = cust_grp[~cust_grp.repm_vin.isin(used_vin_list)]
cust_grp_noused.shape

(1703479, 13)

In [ ]:
cust_grp_noused.head(3)

In [16]:
# Split or partition customer group containing non-used vehicles only
def split(df, name):
    m = cust_grp_noused.unique_cust_grp.unique().tolist()
    n = 200000
    last = 0
    r = 1
    name = name
    path = r'/home/neelima/Service_Latest_Data_Final_Execution'
    while last<len(m):
        df_new = df[df.unique_cust_grp.isin(m[int(last):int(last+n)])]
        name = name + '_' + str(r)
        file = path+'/'+name+'.csv'
        last+=n
        r+=1
        #print(df_new.shape)
        df_new.to_csv(file, encoding='utf-8', index=False)

In [ ]:
split(cust_grp_noused, 'cust_grp_noused_1')

In [17]:
print("cust_grp raw:", cust_grp.shape)
print("cust_grp_noused:", cust_grp_noused.shape)
print("used_cust_grp_1:", used_cust_grp_1.shape)
print("used_cust_grp_2:", used_cust_grp_2.shape)

cust_grp raw: (1924867, 13)
cust_grp_noused: (1703479, 13)
used_cust_grp_1: (232003, 4)
used_cust_grp_2: (187221, 4)


**Service history feature engineering for used cars**

In [18]:
#Remove duplicates from used_car_cust_grp
used_cust_grp_1_dedup = used_cust_grp_1.drop_duplicates(subset=['repm_vin', 'unique_cust_grp'])
used_cust_grp_2_dedup = used_cust_grp_2.drop_duplicates(subset=['repm_vin', 'unique_cust_grp'])

In [19]:
print("used_cust_grp_1_dedup:", used_cust_grp_1_dedup.shape)
print("used_cust_grp_2_dedup:", used_cust_grp_2_dedup.shape)

used_cust_grp_1_dedup: (155554, 4)
used_cust_grp_2_dedup: (127379, 4)


In [20]:
# Reading table used_car_rcrepm
cursor_1.execute('select * from used_car_rcrepm')
used_car_rcrepm = as_pandas(cursor_1)

In [ ]:
used_car_rcrepm_backup = used_car_rcrepm.copy()

In [19]:
print(used_car_rcrepm.shape)
print(used_car_rcrepm.columns)
used_car_rcrepm.head(3)

(33744535, 14)
Index(['repm_dlr_code', 'repm_cust_no', 'repm_ro_no', 'repm_ro_dtime',
       'repm_vin', 'repm_work_type', 'repm_vst_type',
       'repm_bill_total_labr_amt', 'repm_bill_total_othr_amt',
       'repm_bill_total_part_amt', 'vinm_selling_date', 'vinm_sale_date',
       'vinm_model_year', 'vinm_last_milg'],
      dtype='object')


,repm_dlr_code,repm_cust_no,repm_ro_no,repm_ro_dtime,repm_vin,repm_work_type,repm_vst_type,repm_bill_total_labr_amt,repm_bill_total_othr_amt,repm_bill_total_part_amt,vinm_selling_date,vinm_sale_date,vinm_model_year,vinm_last_milg
0,N3208,C2008020258,R201708673,2017-07-29 10:10:50,MALCG51WR6M015343,OC,null,0.00,0.00,0.00,null,20060629,2006,81635
1,W2213,C2008030086,R201610998,2016-11-16 17:14:46,MALCG51WR6M015479,OC,null,0.00,0.00,0.00,null,20061128,2006,75950
2,W2213,C2008030086,R201702566,2017-03-13 18:08:27,MALCG51WR6M015479,OC,null,0.00,0.00,0.00,null,20061128,2006,75950


In [25]:
used_car_rcrepm = used_car_rcrepm.replace("null", np.nan)

In [ ]:
used_car_rcrepm.dtypes

In [23]:
used_car_rcrepm['repm_ro_dtime'] = pd.to_datetime(used_car_rcrepm['repm_ro_dtime'])

In [26]:
used_car_rcrepm['vinm_selling_date'] = pd.to_datetime(used_car_rcrepm['vinm_selling_date'])

In [27]:
used_car_rcrepm['used_car_sale_date'] = pd.to_datetime(used_car_rcrepm['vinm_selling_date'], infer_datetime_format=True)

In [28]:
used_car_rcrepm.shape

(33744535, 15)

In [29]:
#Separate service data for 1st owner vs 2nd owner
used_car_rcrepm_1 = used_car_rcrepm[(used_car_rcrepm['repm_ro_dtime']<used_car_rcrepm['used_car_sale_date'])&
                                   (used_car_rcrepm.repm_work_type!='PD')]

used_car_rcrepm_2 = used_car_rcrepm[(used_car_rcrepm['repm_ro_dtime']>used_car_rcrepm['used_car_sale_date'])&
                                   (used_car_rcrepm.repm_work_type!='PD')]

In [26]:
print("Used_car_rcrepm original:", used_car_rcrepm.shape)
print("Used_car_rcrepm_1:", used_car_rcrepm_1.shape)
print("Used_car_rcrepm_2:", used_car_rcrepm_2.shape)
print("")
print("used_cust_grp_1_dedup:", used_cust_grp_1_dedup.shape)
print("used_cust_grp_2_dedup:", used_cust_grp_2_dedup.shape)

Used_car_rcrepm original: (33744535, 15)
Used_car_rcrepm_1: (909344, 15)
Used_car_rcrepm_2: (644382, 15)

used_cust_grp_1_dedup: (155554, 4)
used_cust_grp_2_dedup: (127379, 4)


In [30]:
#Merge used car service data with used car customer groups
used_car_1_serv_hist = pd.merge(used_car_rcrepm_1, used_cust_grp_1_dedup, how='left', on=['repm_vin'], indicator = True)
used_car_2_serv_hist = pd.merge(used_car_rcrepm_2, used_cust_grp_2_dedup, how='left', on=['repm_vin'], indicator = True)

#Use the above df for used car service feature using the codes below

In [28]:
print("used_car_1_serv_hist :", used_car_1_serv_hist.shape)
print("used_car_2_serv_hist :", used_car_2_serv_hist.shape)

used_car_1_serv_hist : (979526, 19)
used_car_2_serv_hist : (644382, 19)


In [29]:
print("used_car_1_serv_hist :", used_car_1_serv_hist.shape)
used_car_1_serv_hist['_merge'].value_counts()

used_car_1_serv_hist : (979526, 19)


both          598285
left_only     381241
right_only         0
Name: _merge, dtype: int64

In [31]:
print("used_car_2_serv_hist :", used_car_2_serv_hist.shape)
used_car_2_serv_hist['_merge'].value_counts()

used_car_2_serv_hist : (644382, 19)


both          511305
left_only     133077
right_only         0
Name: _merge, dtype: int64

**Testing with the existing variable**

In [32]:
print(cust_grp_noused.shape)
print(cust_grp_noused.used_vin.value_counts())
cust_grp_noused.head(2)

(1703479, 13)
False    1703479
Name: used_vin, dtype: int64


,con_id,con_id_x,con_id_y,phone_no,repm_vin,ser_ph_grp,service_phone,unique_cust_grp,vin_grp,Target,used_1,used_2,used_vin
0,NaN,1-29OZ-1503,1-29OZ-1503,9.89111e+09,MALAA51HR5M700082,1316486,9891112121,0,566852,0,False,False,False
1,NaN,1-29OE-2971,1-29OE-2971,2.5012e+09,MALAA51HR5M700158,235,2501201900,1,566853,0,False,False,False


In [32]:
len(used_vin_list)

127379

In [ ]:
used_car_rcrepm.shape

In [33]:
# Testing on existing variable
serv_noused_test1 = used_car_rcrepm[used_car_rcrepm['used_car_sale_date'].isnull()]
display(serv_noused_test1.shape)

(32171981, 15)

In [34]:
serv_noused_test2 = serv_noused_test1[~serv_noused_test1.repm_vin.isin(used_vin_list)]
print(serv_noused_test2.shape)
serv_noused_test2.head(2)

(32171981, 15)


,repm_dlr_code,repm_cust_no,repm_ro_no,repm_ro_dtime,repm_vin,repm_work_type,repm_vst_type,repm_bill_total_labr_amt,repm_bill_total_othr_amt,repm_bill_total_part_amt,vinm_selling_date,vinm_sale_date,vinm_model_year,vinm_last_milg,used_car_sale_date
0,N3208,C2008020258,R201708673,2017-07-29 10:10:50,MALCG51WR6M015343,OC,NaN,0.00,0.00,0.00,NaT,20060629,2006,81635,NaT
1,W2213,C2008030086,R201610998,2016-11-16 17:14:46,MALCG51WR6M015479,OC,NaN,0.00,0.00,0.00,NaT,20061128,2006,75950,NaT


**Feature engineering for not used customer group for one partition data. The below needs to be repeated for all partiotined data**

In [35]:
#Merge service data with unique customer group
cust_grp_1 = cust_grp_noused
serv_grp_1 = serv_noused_test2

In [36]:
# Check data sanity
print("cust_grp_1.unique_cust_grp.unique:", len(cust_grp_1.unique_cust_grp.unique()))
print("cust_grp_1.repm_vin.unique: ",len(cust_grp_1.repm_vin.unique()))
print("serv_grp_1.repm_vin.unique: ",len(serv_grp_1.repm_vin.unique()))
print("serv_grp_1.shape: ",serv_grp_1.shape)
cust_grp_1_dict = pd.Series(cust_grp_1['unique_cust_grp'].values, index=cust_grp_1['repm_vin']).to_dict()
serv_grp_1['unique_cust_grp'] = serv_grp_1.repm_vin.map(cust_grp_1_dict)
print("serv_grp_1.shape",serv_grp_1.shape)

cust_grp_1.unique_cust_grp.unique: 1244941
cust_grp_1.repm_vin.unique:  1329223
serv_grp_1.repm_vin.unique:  5193039
serv_grp_1.shape:  (32171981, 15)
serv_grp_1.shape (32171981, 16)


In [37]:
grp_1_vin_age = serv_noused_test2
print(grp_1_vin_age.shape)
grp_1_vin_age.head(2)

(32171981, 16)


,repm_dlr_code,repm_cust_no,repm_ro_no,repm_ro_dtime,repm_vin,repm_work_type,repm_vst_type,repm_bill_total_labr_amt,repm_bill_total_othr_amt,repm_bill_total_part_amt,vinm_selling_date,vinm_sale_date,vinm_model_year,vinm_last_milg,used_car_sale_date,unique_cust_grp
0,N3208,C2008020258,R201708673,2017-07-29 10:10:50,MALCG51WR6M015343,OC,NaN,0.00,0.00,0.00,NaT,20060629,2006,81635,NaT,NaN
1,W2213,C2008030086,R201610998,2016-11-16 17:14:46,MALCG51WR6M015479,OC,NaN,0.00,0.00,0.00,NaT,20061128,2006,75950,NaT,NaN


In [38]:
# Merging with car age
#grp_1_vin_age = pd.read_csv('D:/Referral/Cust_grp_splits/serv_noused_1_vin_age.csv', low_memory=False)
vin_age_map = pd.Series(grp_1_vin_age.vinm_model_year.values,index = grp_1_vin_age.repm_vin).to_dict()
serv_grp_1['car_model_year'] = serv_grp_1.repm_vin.map(vin_age_map)

In [39]:
display(serv_grp_1.shape)
display(serv_grp_1.columns)
display(serv_grp_1.car_model_year.isnull().sum())

(32171981, 17)

Index(['repm_dlr_code', 'repm_cust_no', 'repm_ro_no', 'repm_ro_dtime',
       'repm_vin', 'repm_work_type', 'repm_vst_type',
       'repm_bill_total_labr_amt', 'repm_bill_total_othr_amt',
       'repm_bill_total_part_amt', 'vinm_selling_date', 'vinm_sale_date',
       'vinm_model_year', 'vinm_last_milg', 'used_car_sale_date',
       'unique_cust_grp', 'car_model_year'],
      dtype='object')

15355

In [ ]:
serv_grp_1['car_model_year'].dtypes

In [40]:
serv_grp_1['car_model_year'] = pd.to_numeric(serv_grp_1['car_model_year'])

In [41]:
serv_grp_1['Purchase_Age'] = (2020 - serv_grp_1['car_model_year']) #Calculate car age replace 0 years with 1 year
serv_grp_1['Purchase_Age'] = np.where(serv_grp_1['Purchase_Age']==0, 1, serv_grp_1['Purchase_Age'])
serv_grp_1 = serv_grp_1.drop_duplicates(subset=['repm_dlr_code', 'repm_cust_no', 'repm_vin', 'repm_ro_dtime',
                                  'repm_work_type'])

#removed repm milg from original code -- this variable is present in the ser_rcrepm_tb_0_10L files.
#serv_grp_1 = serv_grp_1.drop_duplicates(subset=['repm_dlr_code', 'repm_cust_no', 'repm_vin', 'repm_ro_dtime',
#                                  'repm_work_type', 'repm_milg'])

In [43]:
display(serv_grp_1.shape)
serv_grp_1 = serv_grp_1[serv_grp_1.car_model_year.notnull()] #Remove rows where car age is not present
display(serv_grp_1.shape)

(32153653, 18)

(32138345, 18)

In [44]:
serv_grp_1['repm_ro_dtime'] = pd.to_datetime(serv_grp_1['repm_ro_dtime'])
serv_grp_1 = serv_grp_1.sort_values(['unique_cust_grp', 'repm_vin', 'repm_work_type', 'repm_ro_dtime'], ascending = [True,
                                                                                                                     True,
                                                                                                                    True, False])

/usr/local/src/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [45]:
print(serv_grp_1.shape)
serv_grp_1.head(2)

(32138345, 18)


,repm_dlr_code,repm_cust_no,repm_ro_no,repm_ro_dtime,repm_vin,repm_work_type,repm_vst_type,repm_bill_total_labr_amt,repm_bill_total_othr_amt,repm_bill_total_part_amt,vinm_selling_date,vinm_sale_date,vinm_model_year,vinm_last_milg,used_car_sale_date,unique_cust_grp,car_model_year,Purchase_Age
7995749,N1201,C2005100362,R201932851,2019-11-28 09:18:30,MALAA51HR5M700082,PS,P,5974.34,0.00,7910.51,NaT,20051016,2005,64235,NaT,0,2005.0,15.0
7995751,N1201,C2005100362,R201900002,2019-01-02 08:51:38,MALAA51HR5M700082,PS,NaN,2837.90,0.00,1457.20,NaT,20051016,2005,64235,NaT,0,2005.0,15.0


In [ ]:
display(serv_grp_1.repm_work_type.value_counts())
display(serv_grp_1.head())

In [46]:
serv_grp_1.sort_values(['repm_vin', 'repm_ro_dtime'], ascending = [True, False], inplace = True)
serv_grp_1.head()

,repm_dlr_code,repm_cust_no,repm_ro_no,repm_ro_dtime,repm_vin,repm_work_type,repm_vst_type,repm_bill_total_labr_amt,repm_bill_total_othr_amt,repm_bill_total_part_amt,vinm_selling_date,vinm_sale_date,vinm_model_year,vinm_last_milg,used_car_sale_date,unique_cust_grp,car_model_year,Purchase_Age
10098774,N2212,B2010090010,R201813137,2018-11-24 15:03:46,00000000000037519,RR,NaN,0.00,0.00,0.00,NaT,20000113,1999,200394,NaT,NaN,1999.0,21.0
9351338,N6208,S2012070002,R201605202,2016-05-24 12:23:36,00000000000207962,RR,NaN,0.00,0.00,0.00,NaT,20031125,2003,79425,NaT,NaN,2003.0,17.0
13355686,S5208,B2012040014,R201720814,2017-12-28 10:59:00,00000000000236380,RR,NaN,0.00,0.00,0.00,NaT,20040129,2004,82569,NaT,NaN,2004.0,16.0
27086235,S5219,S2013050004,R202001830,2020-06-24 00:00:00,00000000001117907,RR,NaN,0.00,0.00,0.00,NaT,20120530,2008,27000,NaT,NaN,2008.0,12.0
13150439,N2201,B2011060030,R201806287,2018-07-24 15:28:00,00000000001370274,PS,NaN,0.00,0.00,0.00,NaT,19991018,1999,108377,NaT,NaN,1999.0,21.0


In [47]:
serv_grp_1.reset_index(inplace = True)

In [48]:
serv_grp_1['day_diff'] = serv_grp_1.groupby(['repm_vin'])['repm_ro_dtime'].apply(lambda x: x - x.shift(-1))

In [208]:
serv_grp_1.head(2)

,index,repm_dlr_code,repm_cust_no,repm_ro_no,repm_ro_dtime,repm_vin,repm_work_type,repm_vst_type,repm_bill_total_labr_amt,repm_bill_total_othr_amt,repm_bill_total_part_amt,vinm_selling_date,vinm_sale_date,vinm_model_year,vinm_last_milg,used_car_sale_date,unique_cust_grp,car_model_year,Purchase_Age,day_diff,service_age_vin,repm_estmt_amt,repm_prvs_milg,repm_milg,billable_amount,Diff_estimate_bill,Milg_diff_vin,total_service_count_vin,FS_count_vin,FS_count_vin_norm,PS_count_vin,PS_count_vin_norm,RR_count_vin,RR_count_vin_norm,AR_count_vin,AR_count_vin_norm,OC_count_vin,OC_count_vin_norm,year,Repeat_visit_month,Repeat_visit_three_month,rv_3_count,serv_count_last_one,serv_count_last_three,FS_dlr_count_vin,MS_dlr_count_vin,AR_dlr_count_vin,Total_dlr_count_vin,MS_count_vin,Total_serv_count_vin,FS_dealer_loyalty_vin,MS_dealer_loyalty_vin,AR_dealer_loyalty_vin,Overall_dealer_loyalty_vin,sale_date,sale_year
14713819,7995749,N1201,C2005100362,R201932851,2019-11-28 09:18:30,MALAA51HR5M700082,PS,P,5974.34,0.00,7910.51,NaT,2005-10-16,2005,64235,NaT,0.0,2005.0,15.0,330.0,1239.030208,NaN,NaN,NaN,13884.85,NaN,NaN,8,NaN,NaN,5.0,0.333333,3.0,0.2,NaN,NaN,NaN,NaN,2019,False,False,3.0,2.0,8.0,NaN,1.0,NaN,1,8.0,8,NaN,8.0,NaN,8.0,2005.0,2005.0
14713820,7995751,N1201,C2005100362,R201900002,2019-01-02 08:51:38,MALAA51HR5M700082,PS,NaN,2837.90,0.00,1457.20,NaT,2005-10-16,2005,64235,NaT,0.0,2005.0,15.0,250.0,1239.030208,NaN,NaN,NaN,4295.10,NaN,NaN,8,NaN,NaN,5.0,0.333333,3.0,0.2,NaN,NaN,NaN,NaN,2019,False,False,3.0,2.0,8.0,NaN,1.0,NaN,1,8.0,8,NaN,8.0,NaN,8.0,2005.0,2005.0


In [50]:
serv_grp_1['day_diff'] = serv_grp_1['day_diff'].dt.days
serv_grp_1.head(3)

,index,repm_dlr_code,repm_cust_no,repm_ro_no,repm_ro_dtime,repm_vin,repm_work_type,repm_vst_type,repm_bill_total_labr_amt,repm_bill_total_othr_amt,repm_bill_total_part_amt,vinm_selling_date,vinm_sale_date,vinm_model_year,vinm_last_milg,used_car_sale_date,unique_cust_grp,car_model_year,Purchase_Age,day_diff
0,10098774,N2212,B2010090010,R201813137,2018-11-24 15:03:46,00000000000037519,RR,NaN,0.00,0.00,0.00,NaT,20000113,1999,200394,NaT,NaN,1999.0,21.0,NaN
1,9351338,N6208,S2012070002,R201605202,2016-05-24 12:23:36,00000000000207962,RR,NaN,0.00,0.00,0.00,NaT,20031125,2003,79425,NaT,NaN,2003.0,17.0,NaN
2,13355686,S5208,B2012040014,R201720814,2017-12-28 10:59:00,00000000000236380,RR,NaN,0.00,0.00,0.00,NaT,20040129,2004,82569,NaT,NaN,2004.0,16.0,NaN


In [209]:
serv_grp_1.rename(columns = {'Purchase_Age':'Age_of_car'}, inplace = True)

In [210]:
serv_grp_1.head(3)

,index,repm_dlr_code,repm_cust_no,repm_ro_no,repm_ro_dtime,repm_vin,repm_work_type,repm_vst_type,repm_bill_total_labr_amt,repm_bill_total_othr_amt,repm_bill_total_part_amt,vinm_selling_date,vinm_sale_date,vinm_model_year,vinm_last_milg,used_car_sale_date,unique_cust_grp,car_model_year,Age_of_car,day_diff,service_age_vin,repm_estmt_amt,repm_prvs_milg,repm_milg,billable_amount,Diff_estimate_bill,Milg_diff_vin,total_service_count_vin,FS_count_vin,FS_count_vin_norm,PS_count_vin,PS_count_vin_norm,RR_count_vin,RR_count_vin_norm,AR_count_vin,AR_count_vin_norm,OC_count_vin,OC_count_vin_norm,year,Repeat_visit_month,Repeat_visit_three_month,rv_3_count,serv_count_last_one,serv_count_last_three,FS_dlr_count_vin,MS_dlr_count_vin,AR_dlr_count_vin,Total_dlr_count_vin,MS_count_vin,Total_serv_count_vin,FS_dealer_loyalty_vin,MS_dealer_loyalty_vin,AR_dealer_loyalty_vin,Overall_dealer_loyalty_vin,sale_date,sale_year
14713819,7995749,N1201,C2005100362,R201932851,2019-11-28 09:18:30,MALAA51HR5M700082,PS,P,5974.34,0.00,7910.51,NaT,2005-10-16,2005,64235,NaT,0.0,2005.0,15.0,330.0,1239.030208,NaN,NaN,NaN,13884.85,NaN,NaN,8,NaN,NaN,5.0,0.333333,3.0,0.2,NaN,NaN,NaN,NaN,2019,False,False,3.0,2.0,8.0,NaN,1.0,NaN,1,8.0,8,NaN,8.0,NaN,8.0,2005.0,2005.0
14713820,7995751,N1201,C2005100362,R201900002,2019-01-02 08:51:38,MALAA51HR5M700082,PS,NaN,2837.90,0.00,1457.20,NaT,2005-10-16,2005,64235,NaT,0.0,2005.0,15.0,250.0,1239.030208,NaN,NaN,NaN,4295.10,NaN,NaN,8,NaN,NaN,5.0,0.333333,3.0,0.2,NaN,NaN,NaN,NaN,2019,False,False,3.0,2.0,8.0,NaN,1.0,NaN,1,8.0,8,NaN,8.0,NaN,8.0,2005.0,2005.0
14713822,7995750,N1201,C2005100362,R201804504,2018-02-18 08:52:00,MALAA51HR5M700082,PS,NaN,4652.74,1180.00,3717.26,NaT,2005-10-16,2005,64235,NaT,0.0,2005.0,15.0,135.0,1239.030208,NaN,NaN,NaN,9550.00,NaN,NaN,8,NaN,NaN,5.0,0.333333,3.0,0.2,NaN,NaN,NaN,NaN,2018,False,False,3.0,2.0,8.0,NaN,1.0,NaN,1,8.0,8,NaN,8.0,NaN,8.0,2005.0,2005.0


In [211]:
recent_sale_date = serv_grp_1['vinm_sale_date'].max()
recent_sale_date

In [213]:
old_sale_date = serv_grp_1['vinm_sale_date'].min()
old_sale_date

Timestamp('1900-02-18 00:00:00')

In [214]:
test_date = serv_grp_1[(serv_grp_1['vinm_sale_date'].dt.year <= 2020)]

In [215]:
test_date_max = test_date['vinm_sale_date'].max()

In [216]:
test_date_max

Timestamp('2020-12-31 00:00:00')

In [221]:
from dateutil.relativedelta import relativedelta
test_date['diff_years'] = test_date_max - test_date.vinm_sale_date

/usr/local/src/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [224]:
test_date['diff_years'] = test_date.diff_years/ np.timedelta64(1, 'Y')

/usr/local/src/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [225]:
test_date['diff_years'].head(3)

14713819    15.209073
14713820    15.209073
14713822    15.209073
Name: diff_years, dtype: float64

## Calculated Purchase Age of car

In [200]:
serv_grp_1['vinm_sale_date'] = pd.to_datetime(serv_grp_1['vinm_sale_date'], errors = 'coerce')

In [228]:
test_date['Purchase_age'] = test_date['diff_years'].apply(np.floor)

/usr/local/src/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [229]:
test_date.head(4)

,index,repm_dlr_code,repm_cust_no,repm_ro_no,repm_ro_dtime,repm_vin,repm_work_type,repm_vst_type,repm_bill_total_labr_amt,repm_bill_total_othr_amt,repm_bill_total_part_amt,vinm_selling_date,vinm_sale_date,vinm_model_year,vinm_last_milg,used_car_sale_date,unique_cust_grp,car_model_year,Age_of_car,day_diff,service_age_vin,repm_estmt_amt,repm_prvs_milg,repm_milg,billable_amount,Diff_estimate_bill,Milg_diff_vin,total_service_count_vin,FS_count_vin,FS_count_vin_norm,PS_count_vin,PS_count_vin_norm,RR_count_vin,RR_count_vin_norm,AR_count_vin,AR_count_vin_norm,OC_count_vin,OC_count_vin_norm,year,Repeat_visit_month,Repeat_visit_three_month,rv_3_count,serv_count_last_one,serv_count_last_three,FS_dlr_count_vin,MS_dlr_count_vin,AR_dlr_count_vin,Total_dlr_count_vin,MS_count_vin,Total_serv_count_vin,FS_dealer_loyalty_vin,MS_dealer_loyalty_vin,AR_dealer_loyalty_vin,Overall_dealer_loyalty_vin,sale_date,sale_year,diff_years,Purchase_age
14713819,7995749,N1201,C2005100362,R201932851,2019-11-28 09:18:30,MALAA51HR5M700082,PS,P,5974.34,0.00,7910.51,NaT,2005-10-16,2005,64235,NaT,0.0,2005.0,15.0,330.0,1239.030208,NaN,NaN,NaN,13884.85,NaN,NaN,8,NaN,NaN,5.0,0.333333,3.0,0.2,NaN,NaN,NaN,NaN,2019,False,False,3.0,2.0,8.0,NaN,1.0,NaN,1,8.0,8,NaN,8.0,NaN,8.0,2005.0,2005.0,15.209073,15.0
14713820,7995751,N1201,C2005100362,R201900002,2019-01-02 08:51:38,MALAA51HR5M700082,PS,NaN,2837.90,0.00,1457.20,NaT,2005-10-16,2005,64235,NaT,0.0,2005.0,15.0,250.0,1239.030208,NaN,NaN,NaN,4295.10,NaN,NaN,8,NaN,NaN,5.0,0.333333,3.0,0.2,NaN,NaN,NaN,NaN,2019,False,False,3.0,2.0,8.0,NaN,1.0,NaN,1,8.0,8,NaN,8.0,NaN,8.0,2005.0,2005.0,15.209073,15.0
14713822,7995750,N1201,C2005100362,R201804504,2018-02-18 08:52:00,MALAA51HR5M700082,PS,NaN,4652.74,1180.00,3717.26,NaT,2005-10-16,2005,64235,NaT,0.0,2005.0,15.0,135.0,1239.030208,NaN,NaN,NaN,9550.00,NaN,NaN,8,NaN,NaN,5.0,0.333333,3.0,0.2,NaN,NaN,NaN,NaN,2018,False,False,3.0,2.0,8.0,NaN,1.0,NaN,1,8.0,8,NaN,8.0,NaN,8.0,2005.0,2005.0,15.209073,15.0
14713825,7995746,N1201,C2005100362,R201708463,2017-04-01 08:36:57,MALAA51HR5M700082,PS,NaN,3075.12,0.00,3192.83,NaT,2005-10-16,2005,64235,NaT,0.0,2005.0,15.0,268.0,1239.030208,NaN,NaN,NaN,6267.95,NaN,NaN,8,NaN,NaN,5.0,0.333333,3.0,0.2,NaN,NaN,NaN,NaN,2017,False,False,3.0,2.0,8.0,NaN,1.0,NaN,1,8.0,8,NaN,8.0,NaN,8.0,2005.0,2005.0,15.209073,15.0


In [238]:
test_date.to_csv("age_columns.csv")

MemoryError: 

In [202]:
serv_grp_1['sale_year'] = serv_grp_1['vinm_sale_date'].dt.year

In [51]:
serv_grp_1_x = serv_grp_1.groupby('repm_vin')['repm_ro_dtime'].apply(lambda x: x.max()-x.min()).reset_index().rename(
    columns={'repm_ro_dtime': 'service_age_vin'})

In [ ]:
# Dataframe with car age 
test_date_1_x = serv_grp_1.groupby('repm_vin')['repm_ro_dtime'].apply(lambda x: x.max()-x.min()).reset_index().rename(
    columns={'repm_ro_dtime': 'service_age_vin'})

In [52]:
print(serv_grp_1_x.shape)
serv_grp_1_x.head(3)

(5190368, 2)


,repm_vin,service_age_vin
0,00000000000037519,0 days
1,00000000000207962,0 days
2,00000000000236380,0 days


In [53]:
serv_grp_1_x['service_age_vin'] = serv_grp_1_x['service_age_vin']/np.timedelta64(1,'D')
serv_grp_1_x.head()

,repm_vin,service_age_vin
0,00000000000037519,0.0
1,00000000000207962,0.0
2,00000000000236380,0.0
3,00000000001117907,0.0
4,00000000001370274,0.0


In [54]:
serv_grp_1 = pd.merge(serv_grp_1, serv_grp_1_x, how = 'left', on = ['repm_vin'])
#del(serv_grp_1_x)
serv_grp_1.head(3)

,index,repm_dlr_code,repm_cust_no,repm_ro_no,repm_ro_dtime,repm_vin,repm_work_type,repm_vst_type,repm_bill_total_labr_amt,repm_bill_total_othr_amt,...,vinm_selling_date,vinm_sale_date,vinm_model_year,vinm_last_milg,used_car_sale_date,unique_cust_grp,car_model_year,Purchase_Age,day_diff,service_age_vin
0,10098774,N2212,B2010090010,R201813137,2018-11-24 15:03:46,00000000000037519,RR,NaN,0.00,0.00,...,NaT,20000113,1999,200394,NaT,NaN,1999.0,21.0,NaN,0.0
1,9351338,N6208,S2012070002,R201605202,2016-05-24 12:23:36,00000000000207962,RR,NaN,0.00,0.00,...,NaT,20031125,2003,79425,NaT,NaN,2003.0,17.0,NaN,0.0
2,13355686,S5208,B2012040014,R201720814,2017-12-28 10:59:00,00000000000236380,RR,NaN,0.00,0.00,...,NaT,20040129,2004,82569,NaT,NaN,2004.0,16.0,NaN,0.0


In [69]:
serv_grp_1['service_age_vin'].dtype

dtype('float64')

In [57]:
print(serv_grp_1.shape)
print(serv_grp_1.columns)

(32138345, 21)
Index(['index', 'repm_dlr_code', 'repm_cust_no', 'repm_ro_no', 'repm_ro_dtime',
       'repm_vin', 'repm_work_type', 'repm_vst_type',
       'repm_bill_total_labr_amt', 'repm_bill_total_othr_amt',
       'repm_bill_total_part_amt', 'vinm_selling_date', 'vinm_sale_date',
       'vinm_model_year', 'vinm_last_milg', 'used_car_sale_date',
       'unique_cust_grp', 'car_model_year', 'Purchase_Age', 'day_diff',
       'service_age_vin'],
      dtype='object')


In [21]:
# Reading table refrl_ser_rcrepm_tb_final
cursor_1.execute('select repm_dlr_no, repm_ro_no, repm_vin, repm_cust_no, repm_ro_stat, repm_estmt_no, repm_work_type,  repm_milg, repm_estmt_amt, repm_ro_dtime, repm_bill_total_part_amt, repm_bill_total_labr_amt, repm_bill_total_othr_amt, repm_total_labr_amt, repm_total_part_amt, repm_total_othr_amt, repm_vst_type, repm_vst_type_seq_no, repm_rmkrs, repm_crte_dtime, repm_prvs_milg, repm_used_flag, repm_byng_dlr_code, repm_no_details, repm_ro_stat_new, repm_work_category, repm_dlr_code, repm_exit_milg, repm_file_doc_id, repm_ro_mobile_flag, model from refrl_ser_rcrepm_tb_final where repm_ro_dtime >= "2020-06-01" and repm_ro_dtime <= "2020-12-31"')
refrl_ser_rcrepm_tb = as_pandas(cursor_1)

In [22]:
refrl_ser_rcrepm_tb = refrl_ser_rcrepm_tb.replace("null", np.nan)

In [ ]:
refrl_ser_rcrepm_tb.shape

In [ ]:
refrl_ser_rcrepm_tb.head(3)

In [ ]:
serv_grp_1.columns

In [55]:
serv_grp_1 = pd.merge(serv_grp_1, refrl_ser_rcrepm_tb[['repm_vin','repm_estmt_amt','repm_prvs_milg','repm_milg']], how ='left', on ='repm_vin')

In [ ]:
serv_grp_1.head()

In [ ]:
del(serv_grp_1_x)

In [56]:
def serv_hist(df):
    df['Diff_estimate_bill'] = df['repm_estmt_amt'] - df['billable_amount']
    df['Milg_diff_vin'] = df['repm_milg'] - df['repm_prvs_milg']
    df['total_service_count_vin'] = df.repm_vin.map(df.groupby('repm_vin')['repm_ro_dtime'].nunique())
    df['FS_count_vin'] = df.repm_vin.map(df[(df['repm_work_type']=='FS')|
                                            (df['repm_work_type']=='UF')].groupby('repm_vin')['repm_ro_dtime'].nunique())
    df['FS_count_vin_norm'] = df['FS_count_vin']/df['Purchase_Age']
    df['PS_count_vin'] = df.repm_vin.map(df[(df['repm_work_type']=='PS')].groupby('repm_vin')['repm_ro_dtime'].nunique())
    df['PS_count_vin_norm'] = df['PS_count_vin']/df['Purchase_Age']
    df['RR_count_vin'] = df.repm_vin.map(df[(df['repm_work_type']=='RR')].groupby('repm_vin')['repm_ro_dtime'].nunique())
    df['RR_count_vin_norm'] = df['RR_count_vin']/df['Purchase_Age']
    df['AR_count_vin'] = df.repm_vin.map(df[(df['repm_work_type']=='AR')|
                            (df['repm_work_type']=='RA')].groupby('repm_vin')['repm_ro_dtime'].nunique())
    df['AR_count_vin_norm'] = df['AR_count_vin']/df['Purchase_Age']
    df['OC_count_vin'] = df.repm_vin.map(df[(df['repm_work_type']=='OC')].groupby('repm_vin')['repm_ro_dtime'].nunique())
    df['OC_count_vin_norm'] = df['OC_count_vin']/df['Purchase_Age']
    return df

In [57]:
serv_grp_1['billable_amount'] = serv_grp_1['repm_bill_total_labr_amt'] + serv_grp_1['repm_bill_total_othr_amt'] + serv_grp_1['repm_bill_total_part_amt']

In [58]:
list(serv_grp_1.columns)

['index',
 'repm_dlr_code',
 'repm_cust_no',
 'repm_ro_no',
 'repm_ro_dtime',
 'repm_vin',
 'repm_work_type',
 'repm_vst_type',
 'repm_bill_total_labr_amt',
 'repm_bill_total_othr_amt',
 'repm_bill_total_part_amt',
 'vinm_selling_date',
 'vinm_sale_date',
 'vinm_model_year',
 'vinm_last_milg',
 'used_car_sale_date',
 'unique_cust_grp',
 'car_model_year',
 'Purchase_Age',
 'day_diff',
 'service_age_vin',
 'repm_estmt_amt',
 'repm_prvs_milg',
 'repm_milg',
 'billable_amount']

In [236]:
serv_grp_1['repm_vin']

14713819    MALAA51HR5M700082
14713820    MALAA51HR5M700082
14713822    MALAA51HR5M700082
14713825    MALAA51HR5M700082
14713826    MALAA51HR5M700082
14713821    MALAA51HR5M700082
14713823    MALAA51HR5M700082
14713824    MALAA51HR5M700082
14713913    MALAA51HR5M700158
14713972    MALAA51HR5M700178
14713971    MALAA51HR5M700178
14713973    MALAA51HR5M700178
14713970    MALAA51HR5M700178
14713981    MALAA51HR5M700666
14713980    MALAA51HR5M700666
14713983    MALAA51HR5M700666
14713985    MALAA51HR5M700666
14713986    MALAA51HR5M700666
14713989    MALAA51HR5M700666
14713990    MALAA51HR5M700666
14713991    MALAA51HR5M700666
14713982    MALAA51HR5M700666
14713984    MALAA51HR5M700666
14713987    MALAA51HR5M700666
14713988    MALAA51HR5M700666
14714071    MALAA51HR5M700709
14714072    MALAA51HR5M700709
14714073    MALAA51HR5M700709
14714075    MALAA51HR5M700709
14714074    MALAA51HR5M700709
                  ...        
47293177    MALSW81XSHM013579
47293178    MALSW81XSHM013579
47293179  

In [59]:
serv_grp_1_tst = serv_hist(serv_grp_1)

In [ ]:
print(serv_grp_1_tst.shape)
print(serv_grp_1_tst.columns)
serv_grp_1_tst.head(3)

In [60]:
serv_grp_1_tst = serv_grp_1_tst.sort_values(['unique_cust_grp', 'repm_vin', 'repm_ro_dtime'], ascending = [True, True, False])

In [61]:
serv_grp_1_tst['repm_work_type'].value_counts()

RR    13624352
PS    11554456
FS     9593199
AR     6117478
PD     2867079
OC     2408536
AC     1044105
MT       73226
RM        5939
UF        4383
RA         426
Name: repm_work_type, dtype: int64

In [62]:
serv_grp_1_tst['FS_mlg_diff'] = serv_grp_1_tst[(serv_grp_1_tst['repm_work_type']=='FS')|
                                               (serv_grp_1_tst['repm_work_type']=='UF')].groupby(['repm_vin'])['repm_milg'].apply(lambda x: x - x.shift(-1))

serv_grp_1_tst['PS_mlg_diff'] = serv_grp_1_tst[(serv_grp_1_tst['repm_work_type']=='PS')].groupby(['repm_vin'])['repm_milg'].apply(lambda x: x - x.shift(-1))

serv_grp_1_tst['RR_mlg_diff'] = serv_grp_1_tst[(serv_grp_1_tst['repm_work_type']=='RR')].groupby(['repm_vin'])['repm_milg'].apply(lambda x: x - x.shift(-1))


In [63]:
serv_grp_1_tst['OC_mlg_diff'] = serv_grp_1_tst[(serv_grp_1_tst['repm_work_type']=='OC')].groupby(['repm_vin'])['repm_milg'].apply(lambda x: x - x.shift(-1))

serv_grp_1_tst['AR_mlg_diff'] = serv_grp_1_tst[(serv_grp_1_tst['repm_work_type']=='AR')|
                                             (serv_grp_1_tst['repm_work_type']=='RA')].groupby(['repm_vin'])['repm_milg'].apply(lambda x: x - x.shift(-1))

serv_grp_1_tst['FS_day_diff'] = serv_grp_1_tst[(serv_grp_1_tst['repm_work_type']=='FS')|
                                               (serv_grp_1_tst['repm_work_type']=='UF')].groupby(['repm_vin'])['repm_ro_dtime'].apply(lambda x: x - x.shift(-1))

In [64]:
serv_grp_1_tst['PS_day_diff'] = serv_grp_1_tst[(serv_grp_1_tst['repm_work_type']=='PS')].groupby(['repm_vin'])['repm_ro_dtime'].apply(lambda x: x - x.shift(-1))

serv_grp_1_tst['RR_day_diff'] = serv_grp_1_tst[(serv_grp_1_tst['repm_work_type']=='RR')].groupby(['repm_vin'])['repm_ro_dtime'].apply(lambda x: x - x.shift(-1))

serv_grp_1_tst['OC_day_diff'] = serv_grp_1_tst[(serv_grp_1_tst['repm_work_type']=='OC')].groupby(['repm_vin'])['repm_ro_dtime'].apply(lambda x: x - x.shift(-1))

serv_grp_1_tst['AR_day_diff'] = serv_grp_1_tst[(serv_grp_1_tst['repm_work_type']=='AR')|
                                             (serv_grp_1_tst['repm_work_type']=='RA')].groupby(['repm_vin'])['repm_ro_dtime'].apply(lambda x: x - x.shift(-1))

In [65]:
serv_grp_1_tst['FS_day_diff'] = serv_grp_1_tst['FS_day_diff']/np.timedelta64(1,'D')
serv_grp_1_tst['PS_day_diff'] = serv_grp_1_tst['PS_day_diff']/np.timedelta64(1,'D')
serv_grp_1_tst['RR_day_diff'] = serv_grp_1_tst['RR_day_diff']/np.timedelta64(1,'D')
serv_grp_1_tst['OC_day_diff'] = serv_grp_1_tst['OC_day_diff']/np.timedelta64(1,'D')
serv_grp_1_tst['AR_day_diff'] = serv_grp_1_tst['AR_day_diff']/np.timedelta64(1,'D')

In [66]:
def flat(df):
    df.columns = [' '.join(col).strip() for col in df.columns.values]
    return df

In [70]:
serv_grp_1_tst.dtypes

index                                int64
repm_dlr_code                       object
repm_cust_no                        object
repm_ro_no                          object
repm_ro_dtime               datetime64[ns]
repm_vin                            object
repm_work_type                      object
repm_vst_type                       object
repm_bill_total_labr_amt            object
repm_bill_total_othr_amt            object
repm_bill_total_part_amt            object
vinm_selling_date           datetime64[ns]
vinm_sale_date                      object
vinm_model_year                     object
vinm_last_milg                       int64
used_car_sale_date          datetime64[ns]
unique_cust_grp                     object
car_model_year                     float64
Purchase_Age                       float64
day_diff                           float64
service_age_vin                    float64
repm_estmt_amt                      object
repm_prvs_milg                     float64
repm_milg  

In [71]:
serv_grp_1_tst['billable_amount'] = pd.to_numeric(serv_grp_1_tst['billable_amount'])
serv_grp_1_tst['Diff_estimate_bill'] = pd.to_numeric(serv_grp_1_tst['Diff_estimate_bill'])

In [72]:
serv_feat_noused_1 = serv_grp_1_tst.groupby('unique_cust_grp').agg({'FS_mlg_diff': ['mean', 'median', 'min', 'max'],
                                                             'PS_mlg_diff': ['mean', 'median', 'min', 'max'],
                                                             'RR_mlg_diff': ['mean', 'median', 'min', 'max'],
                                                              'OC_mlg_diff': ['mean', 'median', 'min', 'max'],
                                                             'AR_mlg_diff': ['mean', 'median', 'min', 'max'],
                                                              'Milg_diff_vin':['mean', 'median', 'min', 'max'],
                                                              'billable_amount': ['mean', 'median', 'min', 'max'],
                                                             'Diff_estimate_bill': ['mean', 'median', 'min', 'max'],                                                                    
                                                             'FS_day_diff': ['mean', 'median', 'min', 'max'],
                                                             'PS_day_diff': ['mean', 'median', 'min', 'max'],
                                                             'RR_day_diff': ['mean', 'median', 'min', 'max'],
                                                             'OC_day_diff': ['mean', 'median', 'min', 'max'],
                                                             'AR_day_diff': ['mean', 'median', 'min', 'max'],
                                                             'FS_count_vin_norm':['mean', 'median', 'min', 'max'],
                                                             'PS_count_vin_norm': ['mean', 'median', 'min', 'max'],
                                                             'RR_count_vin_norm': ['mean', 'median', 'min', 'max'],
                                                             'OC_count_vin_norm': ['mean', 'median', 'min', 'max'],
                                                             'AR_count_vin_norm': ['mean', 'median', 'min', 'max'],
                                                             'FS_count_vin':['mean', 'median', 'min', 'max'],
                                                             'PS_count_vin': ['mean', 'median', 'min', 'max'],
                                                             'RR_count_vin': ['mean', 'median', 'min', 'max'],
                                                             'OC_count_vin': ['mean', 'median', 'min', 'max'],
                                                             'AR_count_vin': ['mean', 'median', 'min', 'max'],
                                                             'repm_vin': pd.Series.nunique,
                                                             'Purchase_Age': 'mean',
                                                             'service_age_vin': 'mean'
                                                             }).reset_index()

In [73]:
print(serv_feat_noused_1.shape)
serv_feat_noused_1.head()

(1229214, 96)


unique_cust_grp FS_mlg_diff                PS_mlg_diff                   \
                         mean median min max        mean median  min  max   
0               0         NaN    NaN NaN NaN         NaN    NaN  NaN  NaN   
1               1         NaN    NaN NaN NaN         NaN    NaN  NaN  NaN   
2               2         NaN    NaN NaN NaN         NaN    NaN  NaN  NaN   
3               3         NaN    NaN NaN NaN         0.0    0.0  0.0  0.0   
4               4         NaN    NaN NaN NaN         NaN    NaN  NaN  NaN   

  RR_mlg_diff  ... OC_count_vin           AR_count_vin                 \
         mean  ...       median  min  max         mean median min max   
0         NaN  ...          NaN  NaN  NaN          NaN    NaN NaN NaN   
1         NaN  ...          1.0  1.0  1.0          NaN    NaN NaN NaN   
2         NaN  ...          1.0  1.0  1.0          NaN    NaN NaN NaN   
3         0.0  ...          1.0  1.0  1.0          NaN    NaN NaN NaN   
4         NaN  ...          4.0  4.0  4.0          NaN    NaN NaN NaN   

  repm_vin Purchase_Age service_age_vin  
   nunique         mean            mean  
0        1         15.0     1239.030208  
1        1         15.0        0.000000  
2        1         15.0      389.055081  
3        1         15.0     1574.361065  
4        1         15.0      904.521053  

[5 rows x 96 columns]

In [74]:
serv_feat_noused_1 = flat(serv_feat_noused_1)
#display(serv_feat_noused_1.head())

In [75]:
split(serv_feat_noused_1,'serv_feat_noused')

**Repeat visit and recent visit analysis**

In [76]:
cust_grp_1 = pd.read_csv('/home/neelima/Service_Latest_Data_Final_Execution/cust_grp_noused/cust_grp_noused_1.csv', low_memory=False)

# serv_grp_1 = serv_noused_1
# Replace serv_grp_1 here with serv_noused_1 as we have performed operations on serv_grp_1 in earlier cells
# Hence again reinitiate serv_grp_1 = serv_noused_1
#serv_grp_1 = pd.read_csv('D:/Referral/Cust_grp_splits/serv_noused_1.csv', low_memory=False)

In [77]:
## complete set of code is already execute in above 
#cust_grp_1 = pd.read_csv('D:/Referral/Cust_grp_splits/cust_grp_noused_1.csv', low_memory=False)
#serv_grp_1 = pd.read_csv('D:/Referral/Cust_grp_splits/serv_noused_1.csv', low_memory=False)
cust_grp_1_dict = pd.Series(cust_grp_1['unique_cust_grp'].values, index=cust_grp_1['repm_vin']).to_dict()
serv_grp_1['unique_cust_grp'] = serv_grp_1.repm_vin.map(cust_grp_1_dict)
del(cust_grp_1)
serv_grp_1 = serv_grp_1.drop_duplicates(subset=['repm_dlr_code', 'repm_cust_no', 'repm_vin', 'repm_ro_dtime',
                                  'repm_work_type', 'repm_milg'])
serv_grp_1['repm_ro_dtime'] = pd.to_datetime(serv_grp_1['repm_ro_dtime'])
serv_grp_1 = serv_grp_1.sort_values(['unique_cust_grp', 'repm_vin', 'repm_work_type', 'repm_ro_dtime'], ascending=[True, 
                                                                                                                  True, True,
                                                                                                                  False])
serv_grp_1['year'] = serv_grp_1['repm_ro_dtime'].dt.year

serv_grp_1['day_diff'] = serv_grp_1.sort_values(['repm_vin', 'repm_ro_dtime'], ascending=[True,False]).groupby(
    ['repm_vin'])['repm_ro_dtime'].apply(lambda x: x - x.shift(-1))
serv_grp_1['day_diff'] = serv_grp_1['day_diff'].dt.days

In [ ]:
display(serv_grp_1.columns)

In [78]:
split(serv_grp_1, "serv_grp")

/usr/local/src/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [80]:
serv_grp_1['Repeat_visit_month'] = np.where((serv_grp_1['day_diff']<30), True, False)

In [81]:
serv_grp_1['Repeat_visit_three_month'] = np.where((serv_grp_1['day_diff']<100), True, False)

In [82]:
display(serv_grp_1['Repeat_visit_month'].value_counts())
display(serv_grp_1['Repeat_visit_three_month'].value_counts())

False    25814130
True     20876464
Name: Repeat_visit_month, dtype: int64

True     27929487
False    18761107
Name: Repeat_visit_three_month, dtype: int64

In [83]:
serv_rv_three_flags = serv_grp_1[serv_grp_1['Repeat_visit_three_month']==True].groupby('unique_cust_grp')['repm_ro_dtime'].count().reset_index()
serv_count_one_yr = serv_grp_1[serv_grp_1['repm_ro_dtime']>='2018-06-01'].groupby('unique_cust_grp')['repm_ro_dtime'].count().reset_index()
serv_count_three_yr = serv_grp_1[serv_grp_1['repm_ro_dtime']>='2016-06-01'].groupby('unique_cust_grp')['repm_ro_dtime'].count().reset_index()

In [84]:
print(serv_rv_three_flags.shape)
serv_rv_three_flags.tail(3)

(151484, 2)


,unique_cust_grp,repm_ro_dtime
151481,200047.0,10
151482,200048.0,11
151483,200049.0,4


In [85]:
print(serv_count_one_yr.shape)
serv_count_one_yr.tail(3)

(176574, 2)


,unique_cust_grp,repm_ro_dtime
176571,200047.0,13
176572,200048.0,8
176573,200049.0,6


In [86]:
print(serv_count_three_yr.shape)
serv_count_three_yr.tail(3)

(199905, 2)


,unique_cust_grp,repm_ro_dtime
199902,200047.0,20
199903,200048.0,14
199904,200049.0,6


In [87]:
serv_rv_yrly_count = serv_grp_1[serv_grp_1['Repeat_visit_month']==True].groupby(['unique_cust_grp', 'car_model_year'])['repm_ro_dtime'].count().reset_index()

serv_rv_one_yr_count = serv_rv_yrly_count.groupby('unique_cust_grp').agg({'repm_ro_dtime': ['mean', 'max', 'median', 'min']}).reset_index()
serv_day_diff_count = serv_grp_1.groupby('unique_cust_grp').agg({'day_diff': ['mean', 'max', 'median', 'min']}).reset_index()

In [88]:
print(serv_rv_yrly_count.shape)
serv_rv_yrly_count.tail(3)

(125914, 3)


,unique_cust_grp,car_model_year,repm_ro_dtime
125911,200047.0,2014.0,8
125912,200048.0,2017.0,8
125913,200049.0,2017.0,1


In [89]:
print(serv_rv_one_yr_count.shape)
serv_rv_one_yr_count.tail(3)

(116181, 5)


unique_cust_grp repm_ro_dtime               
                                mean max median min
116178        200047.0           8.0   8    8.0   8
116179        200048.0           8.0   8    8.0   8
116180        200049.0           1.0   1    1.0   1

In [90]:
print(serv_day_diff_count.shape)
serv_day_diff_count.tail(3)

(199935, 5)


unique_cust_grp    day_diff                    
                              mean    max median   min
199932        200047.0  122.333333  356.0   59.5   0.0
199933        200048.0   41.230769  113.0   19.0   2.0
199934        200049.0   61.400000  120.0   59.0  26.0

In [91]:
def flat(df):
    df.columns = [' '.join(col).strip() for col in df.columns.values]
    return df

In [92]:
serv_rv_one_yr_count = flat(serv_rv_one_yr_count)
serv_rv_one_yr_count.tail(3)

,unique_cust_grp,repm_ro_dtime mean,repm_ro_dtime max,repm_ro_dtime median,repm_ro_dtime min
116178,200047.0,8.0,8,8.0,8
116179,200048.0,8.0,8,8.0,8
116180,200049.0,1.0,1,1.0,1


In [93]:
serv_day_diff_count = flat(serv_day_diff_count)
serv_day_diff_count.tail(3)

,unique_cust_grp,day_diff mean,day_diff max,day_diff median,day_diff min
199932,200047.0,122.333333,356.0,59.5,0.0
199933,200048.0,41.230769,113.0,19.0,2.0
199934,200049.0,61.400000,120.0,59.0,26.0


In [94]:
serv_rv_three_flags_dict = pd.Series(serv_rv_three_flags['repm_ro_dtime'].values, index=serv_rv_three_flags['unique_cust_grp']).to_dict()
serv_count_one_yr_dict = pd.Series(serv_count_one_yr['repm_ro_dtime'].values, index=serv_count_one_yr['unique_cust_grp']).to_dict()
serv_count_three_yr_dict = pd.Series(serv_count_three_yr['repm_ro_dtime'].values, index=serv_count_three_yr['unique_cust_grp']).to_dict()

In [95]:
#serv_grp_1['rv_3_count'] = serv_grp_1.unique_cust_grp.map(serv_rv_one_flags_dict).fillna(0)
serv_grp_1['rv_3_count'] = serv_grp_1.unique_cust_grp.map(serv_rv_three_flags_dict).fillna(0)
serv_grp_1['serv_count_last_one'] = serv_grp_1.unique_cust_grp.map(serv_count_one_yr_dict).fillna(0)
serv_grp_1['serv_count_last_three'] = serv_grp_1.unique_cust_grp.map(serv_count_three_yr_dict).fillna(0)

In [96]:
serv_rv_feat_1 = serv_grp_1.drop_duplicates(subset=['unique_cust_grp'])[['unique_cust_grp', 'rv_3_count', 'serv_count_last_one','serv_count_last_three']]

In [97]:
print(serv_rv_feat_1.shape)
serv_rv_feat_1.tail(3)

(199936, 4)


,unique_cust_grp,rv_3_count,serv_count_last_one,serv_count_last_three
1746946,200048.0,11.0,8.0,14.0
1746954,200049.0,4.0,6.0,6.0
0,NaN,0.0,0.0,0.0


In [98]:
serv_rv_feat_2 = pd.merge(serv_rv_feat_1, serv_rv_one_yr_count, how='left', on=['unique_cust_grp']).rename(columns={
    'repm_ro_dtime mean': 'yearly_rv_3_count mean', 'repm_ro_dtime max': 'yearly_rv_3_count max',
    'repm_ro_dtime min': 'yearly_rv_3_count min', 'repm_ro_dtime median': 'yearly_rv_3_count median'
}).fillna(0)

In [99]:
print(serv_rv_feat_2.shape)
serv_rv_feat_2.tail(3)

(199936, 8)


,unique_cust_grp,rv_3_count,serv_count_last_one,serv_count_last_three,yearly_rv_3_count mean,yearly_rv_3_count max,yearly_rv_3_count median,yearly_rv_3_count min
199933,200048.0,11.0,8.0,14.0,8.0,8.0,8.0,8.0
199934,200049.0,4.0,6.0,6.0,1.0,1.0,1.0,1.0
199935,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [100]:
serv_rv_feat = pd.merge(serv_rv_feat_2, serv_day_diff_count, how='left', on=['unique_cust_grp'])

In [101]:
print(serv_rv_feat.shape)
serv_rv_feat.tail(3)

(199936, 12)


,unique_cust_grp,rv_3_count,serv_count_last_one,serv_count_last_three,yearly_rv_3_count mean,yearly_rv_3_count max,yearly_rv_3_count median,yearly_rv_3_count min,day_diff mean,day_diff max,day_diff median,day_diff min
199933,200048.0,11.0,8.0,14.0,8.0,8.0,8.0,8.0,41.230769,113.0,19.0,2.0
199934,200049.0,4.0,6.0,6.0,1.0,1.0,1.0,1.0,61.400000,120.0,59.0,26.0
199935,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,176.714286,330.0,135.0,67.0


In [102]:
serv_rv_feat.to_csv('rv_feat_noused_1.csv', index = False, encoding='utf-8')

In [104]:
serv_rv_feat.to_csv('rv_feat_noused_1.dsv', index = False, encoding='utf-8')

**Service dealer history**

In [106]:
cust_grp_1 = pd.read_csv('cust_grp_noused_1.csv')

/usr/local/src/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (3,6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [107]:
cust_grp_1_dict = pd.Series(cust_grp_1['unique_cust_grp'].values, index=cust_grp_1['repm_vin']).to_dict()
serv_grp_1['unique_cust_grp'] = serv_grp_1.repm_vin.map(cust_grp_1_dict)

In [108]:
serv_grp_1 = serv_grp_1.drop_duplicates(subset=['repm_dlr_code', 'repm_cust_no', 'repm_vin', 'repm_ro_dtime',
                                  'repm_work_type', 'repm_milg'])

In [109]:
display(serv_grp_1.shape)

(46690594, 44)

In [110]:
serv_grp_1['repm_ro_dtime'] = pd.to_datetime(serv_grp_1['repm_ro_dtime'])

In [111]:
serv_grp_1 = serv_grp_1.sort_values(['unique_cust_grp', 'repm_vin', 'repm_work_type', 'repm_ro_dtime'], ascending=[True, 
                                                                                                                  True, True,
                                                                                                                      False])

In [112]:
def dlr_count(df):
    df['FS_dlr_count_vin'] = df.repm_vin.map(df[(df['repm_work_type']=='FS')|
                                            (df['repm_work_type']=='UF')].groupby('repm_vin')['repm_dlr_code'].nunique())
    
    df['MS_dlr_count_vin'] = df.repm_vin.map(df[(df['repm_work_type']=='PS')|
                                            (df['repm_work_type']=='RR')|
                                            (df['repm_work_type']=='RM')].groupby('repm_vin')['repm_dlr_code'].nunique())
    df['AR_dlr_count_vin'] = df.repm_vin.map(df[(df['repm_work_type']=='AR')|
                                            (df['repm_work_type']=='RA')].groupby('repm_vin')['repm_dlr_code'].nunique())
    df['Total_dlr_count_vin'] = df.repm_vin.map(df.groupby('repm_vin')['repm_dlr_code'].nunique())
    df['FS_count_vin'] = df.repm_vin.map(df[(df['repm_work_type']=='FS')|
                                            (df['repm_work_type']=='UF')].groupby('repm_vin')['repm_ro_dtime'].nunique())
    df['MS_count_vin'] = df.repm_vin.map(df[(df['repm_work_type']=='PS')|
                            (df['repm_work_type']=='RR')|
                           (df['repm_work_type']=='RM')].groupby('repm_vin')['repm_ro_dtime'].nunique())
    df['AR_count_vin'] = df.repm_vin.map(df[(df['repm_work_type']=='AR')|
                            (df['repm_work_type']=='RA')].groupby('repm_vin')['repm_ro_dtime'].nunique())    
    df['Total_serv_count_vin'] = df.repm_vin.map(df.groupby('repm_vin')['repm_ro_dtime'].nunique())
    df['FS_dealer_loyalty_vin'] = df['FS_count_vin']/df['FS_dlr_count_vin']
    df['MS_dealer_loyalty_vin'] = df['MS_count_vin']/df['MS_dlr_count_vin']
    df['AR_dealer_loyalty_vin'] = df['AR_count_vin']/df['AR_dlr_count_vin']
    df['Overall_dealer_loyalty_vin'] = df['Total_serv_count_vin']/df['Total_dlr_count_vin']
    
    return df

In [113]:
max_service_dlr = serv_grp_1.groupby('unique_cust_grp')['repm_dlr_code'].apply(max).reset_index().rename(
columns={'repm_dlr_code': 'Max_service_dlr_code'})

In [114]:
print(max_service_dlr.shape)
max_service_dlr.tail(3)

(199935, 2)


,unique_cust_grp,Max_service_dlr_code
199932,200047.0,N3437
199933,200048.0,W4401
199934,200049.0,N4229


In [115]:
max_dlr_dict = pd.Series(max_service_dlr['Max_service_dlr_code'].values, index=max_service_dlr['unique_cust_grp']).to_dict()
serv_grp_1 = dlr_count(serv_grp_1)

In [116]:
print(serv_grp_1.shape)
print(serv_grp_1.columns)
serv_grp_1.head()

(46690594, 54)
Index(['index', 'repm_dlr_code', 'repm_cust_no', 'repm_ro_no', 'repm_ro_dtime',
       'repm_vin', 'repm_work_type', 'repm_vst_type',
       'repm_bill_total_labr_amt', 'repm_bill_total_othr_amt',
       'repm_bill_total_part_amt', 'vinm_selling_date', 'vinm_sale_date',
       'vinm_model_year', 'vinm_last_milg', 'used_car_sale_date',
       'unique_cust_grp', 'car_model_year', 'Purchase_Age', 'day_diff',
       'service_age_vin', 'repm_estmt_amt', 'repm_prvs_milg', 'repm_milg',
       'billable_amount', 'Diff_estimate_bill', 'Milg_diff_vin',
       'total_service_count_vin', 'FS_count_vin', 'FS_count_vin_norm',
       'PS_count_vin', 'PS_count_vin_norm', 'RR_count_vin',
       'RR_count_vin_norm', 'AR_count_vin', 'AR_count_vin_norm',
       'OC_count_vin', 'OC_count_vin_norm', 'year', 'Repeat_visit_month',
       'Repeat_visit_three_month', 'rv_3_count', 'serv_count_last_one',
       'serv_count_last_three', 'FS_dlr_count_vin', 'MS_dlr_count_vin',
       'AR_dlr_count_v

,index,repm_dlr_code,repm_cust_no,repm_ro_no,repm_ro_dtime,repm_vin,repm_work_type,repm_vst_type,repm_bill_total_labr_amt,repm_bill_total_othr_amt,...,FS_dlr_count_vin,MS_dlr_count_vin,AR_dlr_count_vin,Total_dlr_count_vin,MS_count_vin,Total_serv_count_vin,FS_dealer_loyalty_vin,MS_dealer_loyalty_vin,AR_dealer_loyalty_vin,Overall_dealer_loyalty_vin
14713819,7995749,N1201,C2005100362,R201932851,2019-11-28 09:18:30,MALAA51HR5M700082,PS,P,5974.34,0.00,...,NaN,1.0,NaN,1,8.0,8,NaN,8.0,NaN,8.0
14713820,7995751,N1201,C2005100362,R201900002,2019-01-02 08:51:38,MALAA51HR5M700082,PS,NaN,2837.90,0.00,...,NaN,1.0,NaN,1,8.0,8,NaN,8.0,NaN,8.0
14713822,7995750,N1201,C2005100362,R201804504,2018-02-18 08:52:00,MALAA51HR5M700082,PS,NaN,4652.74,1180.00,...,NaN,1.0,NaN,1,8.0,8,NaN,8.0,NaN,8.0
14713825,7995746,N1201,C2005100362,R201708463,2017-04-01 08:36:57,MALAA51HR5M700082,PS,NaN,3075.12,0.00,...,NaN,1.0,NaN,1,8.0,8,NaN,8.0,NaN,8.0
14713826,7995752,N1201,C2005100362,R201616534,2016-07-07 08:35:00,MALAA51HR5M700082,PS,NaN,2969.32,0.00,...,NaN,1.0,NaN,1,8.0,8,NaN,8.0,NaN,8.0


In [117]:
split(serv_grp_1,"serv_grp_1_bfr_dlr")

In [119]:
serv_grp_1_dlr = serv_grp_1.groupby('unique_cust_grp').agg({'FS_dlr_count_vin': ['mean', 'median', 'min', 'max'],
                                                             'MS_dlr_count_vin': ['mean', 'median', 'min', 'max'],
                                                             'AR_dlr_count_vin': ['mean', 'median', 'min', 'max'],
                                                             'repm_dlr_code': pd.Series.nunique,
                                                              'Total_dlr_count_vin': ['mean', 'median', 'min', 'max'],
                                                              'FS_dealer_loyalty_vin': ['mean', 'median', 'min', 'max'],
                                                               'MS_dealer_loyalty_vin': ['mean', 'median', 'min', 'max'],
                                                                'AR_dealer_loyalty_vin': ['mean', 'median', 'min', 'max'],
                                                                'Overall_dealer_loyalty_vin': ['mean', 'median', 'min', 'max'],
                                                             'repm_vin': pd.Series.nunique,
                                                              'repm_ro_dtime': pd.Series.nunique,}).reset_index().rename(
columns={'repm_dlr_code': 'Total_dealer_count', 'repm_ro_dtime': 'Total_service_count'})

In [120]:
print(serv_grp_1_dlr.shape)
print(serv_grp_1_dlr.columns)
serv_grp_1_dlr.head()

(199935, 36)
MultiIndex(levels=[['AR_dealer_loyalty_vin', 'AR_dlr_count_vin', 'FS_dealer_loyalty_vin', 'FS_dlr_count_vin', 'MS_dealer_loyalty_vin', 'MS_dlr_count_vin', 'Overall_dealer_loyalty_vin', 'Total_dealer_count', 'Total_dlr_count_vin', 'Total_service_count', 'repm_vin', 'unique_cust_grp'], ['', 'max', 'mean', 'median', 'min', 'nunique']],
           codes=[[11, 3, 3, 3, 3, 5, 5, 5, 5, 1, 1, 1, 1, 7, 8, 8, 8, 8, 2, 2, 2, 2, 4, 4, 4, 4, 0, 0, 0, 0, 6, 6, 6, 6, 10, 9], [0, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4, 1, 5, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4, 1, 5, 5]])


unique_cust_grp FS_dlr_count_vin                MS_dlr_count_vin         \
                              mean median min max             mean median   
0             0.0              NaN    NaN NaN NaN              1.0    1.0   
1             1.0              NaN    NaN NaN NaN              NaN    NaN   
2             2.0              NaN    NaN NaN NaN              1.0    1.0   
3             3.0              NaN    NaN NaN NaN              2.0    2.0   
4             4.0              NaN    NaN NaN NaN              1.0    1.0   

            AR_dlr_count_vin  ... AR_dealer_loyalty_vin                 \
   min  max             mean  ...                  mean median min max   
0  1.0  1.0              NaN  ...                   NaN    NaN NaN NaN   
1  NaN  NaN              NaN  ...                   NaN    NaN NaN NaN   
2  1.0  1.0              NaN  ...                   NaN    NaN NaN NaN   
3  2.0  2.0              NaN  ...                   NaN    NaN NaN NaN   
4  1.0  1.0              NaN  ...                   NaN    NaN NaN NaN   

  Overall_dealer_loyalty_vin                  repm_vin Total_service_count  
                        mean median  min  max  nunique             nunique  
0                        8.0    8.0  8.0  8.0        1                   8  
1                        1.0    1.0  1.0  1.0        1                   1  
2                        2.0    2.0  2.0  2.0        1                   4  
3                        6.0    6.0  6.0  6.0        1                  12  
4                        2.5    2.5  2.5  2.5        1                   5  

[5 rows x 36 columns]

In [121]:
def flat(df):
    df.columns = [' '.join(col).strip() for col in df.columns.values]
    return df

In [122]:
serv_grp_1_dlr = flat(serv_grp_1_dlr)

In [123]:
print(serv_grp_1_dlr.shape)
print(serv_grp_1_dlr.columns)
serv_grp_1_dlr.head()

(199935, 36)
Index(['unique_cust_grp', 'FS_dlr_count_vin mean', 'FS_dlr_count_vin median',
       'FS_dlr_count_vin min', 'FS_dlr_count_vin max', 'MS_dlr_count_vin mean',
       'MS_dlr_count_vin median', 'MS_dlr_count_vin min',
       'MS_dlr_count_vin max', 'AR_dlr_count_vin mean',
       'AR_dlr_count_vin median', 'AR_dlr_count_vin min',
       'AR_dlr_count_vin max', 'Total_dealer_count nunique',
       'Total_dlr_count_vin mean', 'Total_dlr_count_vin median',
       'Total_dlr_count_vin min', 'Total_dlr_count_vin max',
       'FS_dealer_loyalty_vin mean', 'FS_dealer_loyalty_vin median',
       'FS_dealer_loyalty_vin min', 'FS_dealer_loyalty_vin max',
       'MS_dealer_loyalty_vin mean', 'MS_dealer_loyalty_vin median',
       'MS_dealer_loyalty_vin min', 'MS_dealer_loyalty_vin max',
       'AR_dealer_loyalty_vin mean', 'AR_dealer_loyalty_vin median',
       'AR_dealer_loyalty_vin min', 'AR_dealer_loyalty_vin max',
       'Overall_dealer_loyalty_vin mean', 'Overall_dealer_loyalty_vi

,unique_cust_grp,FS_dlr_count_vin mean,FS_dlr_count_vin median,FS_dlr_count_vin min,FS_dlr_count_vin max,MS_dlr_count_vin mean,MS_dlr_count_vin median,MS_dlr_count_vin min,MS_dlr_count_vin max,AR_dlr_count_vin mean,...,AR_dealer_loyalty_vin mean,AR_dealer_loyalty_vin median,AR_dealer_loyalty_vin min,AR_dealer_loyalty_vin max,Overall_dealer_loyalty_vin mean,Overall_dealer_loyalty_vin median,Overall_dealer_loyalty_vin min,Overall_dealer_loyalty_vin max,repm_vin nunique,Total_service_count nunique
0,0.0,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,8.0,8.0,8.0,8.0,1,8
1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1,1
2,2.0,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,2.0,2.0,2.0,2.0,1,4
3,3.0,NaN,NaN,NaN,NaN,2.0,2.0,2.0,2.0,NaN,...,NaN,NaN,NaN,NaN,6.0,6.0,6.0,6.0,1,12
4,4.0,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,2.5,2.5,2.5,2.5,1,5


In [124]:
serv_grp_1_dlr['Max_service_dlr_code'] = serv_grp_1_dlr.unique_cust_grp.map(max_dlr_dict)

In [125]:
print(serv_grp_1_dlr.shape)
print(serv_grp_1_dlr.columns[35:])

(199935, 37)
Index(['Total_service_count nunique', 'Max_service_dlr_code'], dtype='object')


In [127]:
serv_grp_1_dlr.to_csv('/home/neelima/Service_Latest_Data_Final_Execution/ref/Cust_grp_splits/dlr_feat/serv_dlr_feat_1.dsv', encoding='utf-8', index=False)

In [128]:
serv_grp_1_dlr.to_csv('/home/neelima/Service_Latest_Data_Final_Execution/ref/Cust_grp_splits/dlr_feat/serv_dlr_feat_1.csv', encoding='utf-8', index=False)

In [129]:
cust_grp_1.to_csv('/home/neelima/Service_Latest_Data_Final_Execution/cust_grp_1.csv', encoding='utf-8', index=False)

In [130]:
cust_grp_noused.to_csv('/home/neelima/Service_Latest_Data_Final_Execution/cust_grp_noused.csv', encoding='utf-8', index=False)

**Service marketing campaigns feature extraction**

In [132]:
# Original Code
target_cust_grp = pd.read_csv('Cust_grp_master_final_1.csv', low_memory=False)

In [133]:
print(target_cust_grp.shape)
print(target_cust_grp.columns)
target_cust_grp.head(3)

(1924867, 10)
Index(['con_id', 'con_id_x', 'con_id_y', 'phone_no', 'repm_vin', 'ser_ph_grp',
       'service_phone', 'unique_cust_grp', 'vin_grp', 'Target'],
      dtype='object')


,con_id,con_id_x,con_id_y,phone_no,repm_vin,ser_ph_grp,service_phone,unique_cust_grp,vin_grp,Target
0,NaN,1-29OZ-1503,1-29OZ-1503,9891112121,MALAA51HR5M700082,1316486,9891112121,0.0,566852,0
1,NaN,1-29OE-2971,1-29OE-2971,2501201900,MALAA51HR5M700158,235,2501201900,1.0,566853,0
2,NaN,1-2A9X-3345,1-2A9X-3345,9994155040,MALAA51HR5M700178,1567815,9994155040,2.0,566854,0


In [134]:
camp_data = pd.read_csv('serv_marketing_data.csv') #Camp data with camp counts across uniques cust grp

/usr/local/src/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [135]:
print(camp_data.shape)
print(camp_data.columns)
camp_data.head(3)

(4087599, 13)
Index(['RN', 'ROW_ID', 'NAME', 'EMAIL_ADDR', 'CREATED', 'CON_PER_ID',
       'CAMP_CON_NUM', 'CAMP_LD_WAVE_ID', 'MSG_BNCD_REASON_CD',
       'MSG_BNCD_STAT_CD', 'STAT_CD', 'RO_VIN', 'CUSTOMER_PHONE'],
      dtype='object')


,RN,ROW_ID,NAME,EMAIL_ADDR,CREATED,CON_PER_ID,CAMP_CON_NUM,CAMP_LD_WAVE_ID,MSG_BNCD_REASON_CD,MSG_BNCD_STAT_CD,STAT_CD,RO_VIN,CUSTOMER_PHONE
0,1,1-VYE9WV,Monsoon Tips Emailer June 2018,olacabsindiahyd@gmail.com,2018-06-30,1-L75I-701,1-1952460984,1-WA4PT3,NaN,NaN,NaN,MALA741DLJM307106,7.99797e+09
1,2,1-RUISW7,Road Safety Emailer April 2018,mk@gmail.com,2018-04-28,1-DRZU-2745,1-SEDHI9,1-SE11TR,Hard,MAILBOX UNAVAILABLE(MAILBOX NO,Bounced,MALA851CLGM468466,9.4732e+09
2,3,1-VYE9WV,Monsoon Tips Emailer June 2018,sankhya@gmail.com,2018-07-02,1-F5CA-74,1-1961856399,1-WFW762,NaN,,Sent,MALC181CLGM138708,9.99943e+09


In [136]:
target_cust_grp.unique_cust_grp = target_cust_grp.unique_cust_grp.apply(str)

In [137]:
test = pd.merge(target_cust_grp, camp_data, left_on ="repm_vin",  right_on= "RO_VIN", indicator= True, how= "outer")
test['_merge'].value_counts()

both          2622242
right_only    2096866
left_only      817445
Name: _merge, dtype: int64

In [138]:
#testing 
camp_data = test[test["_merge"]=="both"]
print(camp_data.shape)
print(camp_data.columns)
camp_data.head(3)

(2622242, 24)
Index(['con_id', 'con_id_x', 'con_id_y', 'phone_no', 'repm_vin', 'ser_ph_grp',
       'service_phone', 'unique_cust_grp', 'vin_grp', 'Target', 'RN', 'ROW_ID',
       'NAME', 'EMAIL_ADDR', 'CREATED', 'CON_PER_ID', 'CAMP_CON_NUM',
       'CAMP_LD_WAVE_ID', 'MSG_BNCD_REASON_CD', 'MSG_BNCD_STAT_CD', 'STAT_CD',
       'RO_VIN', 'CUSTOMER_PHONE', '_merge'],
      dtype='object')


,con_id,con_id_x,con_id_y,phone_no,repm_vin,ser_ph_grp,service_phone,unique_cust_grp,vin_grp,Target,...,CREATED,CON_PER_ID,CAMP_CON_NUM,CAMP_LD_WAVE_ID,MSG_BNCD_REASON_CD,MSG_BNCD_STAT_CD,STAT_CD,RO_VIN,CUSTOMER_PHONE,_merge
0,NaN,1-29OZ-1503,1-29OZ-1503,9891112121,MALAA51HR5M700082,1316486,9891112121,0.0,566852,0.0,...,2018-11-24,1-29OZ-1503,1-2651293805,1-17SGHYJ,NaN,NaN,NaN,MALAA51HR5M700082,9891112121,both
1,NaN,1-29OZ-1503,1-29OZ-1503,9891112121,MALAA51HR5M700082,1316486,9891112121,0.0,566852,0.0,...,2019-04-18,1-29OZ-1503,128025,1-1F3BYXD,Hard,MAILBOX UNAVAILABLE(MAILBOX NO,Bounced,MALAA51HR5M700082,9.89111e+09,both
4,NaN,1-2B7U-3041,1-2B7U-3041,9880977808,MALAA51HR5M700666,672422,9448346265,3.0,566855,0.0,...,2018-11-24,1-2B7U-3041,1-2651590041,1-17SGHYJ,NaN,,Sent,MALAA51HR5M700666,9.88098e+09,both


In [139]:
camp_data['unique_cust_grp'] = camp_data['unique_cust_grp'].apply(str)

/usr/local/src/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [140]:
target_cust_grp["Target_final"]= target_cust_grp["Target"]
test_0 = target_cust_grp[target_cust_grp.Target_final==0].unique_cust_grp.unique().tolist() #Create dict to map target values to camp data

In [141]:
camp_data['Target_binary'] = np.where(camp_data['unique_cust_grp'].isin(test_0), 0, 1) #map target values to camp data

/usr/local/src/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [142]:
#Data pre-processing
camp_data_dedup = camp_data.drop_duplicates(subset=['CREATED', 'CON_PER_ID', 'CAMP_CON_NUM', 'CAMP_LD_WAVE_ID','MSG_BNCD_REASON_CD', 'MSG_BNCD_STAT_CD', 'STAT_CD'])

In [143]:
camp_data_dedup.shape

(1978378, 25)

In [144]:
camp_data_dedup['stat_cd'] = camp_data_dedup['STAT_CD'].fillna('NA')

/usr/local/src/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [145]:
camp_data_dedup = camp_data_dedup[['ROW_ID', 'unique_cust_grp', 'repm_vin', 'STAT_CD', 'CREATED']]

In [146]:
camp_data_feat = camp_data_dedup.groupby(['unique_cust_grp']).agg({'ROW_ID': pd.Series.nunique,'repm_vin': pd.Series.nunique}).reset_index() # Total camp count

In [147]:
#Feature extraction test
feat_1 = camp_data_dedup[camp_data_dedup.STAT_CD =='Bounced'].groupby(['unique_cust_grp']).agg({'ROW_ID': pd.Series.nunique}).reset_index().rename(
    columns={'ROW_ID': 'Bounced_count'}) #Bounced email count
print(feat_1.shape)
print(feat_1.columns)
feat_1.head(2)

(296372, 2)
Index(['unique_cust_grp', 'Bounced_count'], dtype='object')


,unique_cust_grp,Bounced_count
0,0.0,1
1,1000001.0,1


In [148]:
feat_2 = camp_data_dedup[camp_data_dedup.STAT_CD=='NA'].groupby(['unique_cust_grp']).agg({'ROW_ID': pd.Series.nunique}).reset_index().rename(
    columns={'row_id': 'NA_count'}) #Email count with no status
print(feat_2.shape)
print(feat_2.columns)
feat_2.head(2)

(0, 2)
Index(['unique_cust_grp', 'ROW_ID'], dtype='object')


,unique_cust_grp,ROW_ID


In [149]:
feat_3 = camp_data_dedup[camp_data_dedup.STAT_CD=='Sent'].groupby(['unique_cust_grp']).agg({'ROW_ID': pd.Series.nunique}).reset_index().rename(
    columns={'row_id': 'Sent'}) #Sent email count
print(feat_3.shape)
print(feat_3.columns)
feat_3.head(2)

(429248, 2)
Index(['unique_cust_grp', 'ROW_ID'], dtype='object')


,unique_cust_grp,ROW_ID
0,100000.0,1
1,1000000.0,1


In [150]:
# Merge sent, bounced and NA camp count to the main data
camp_data_feat = pd.merge(camp_data_feat, feat_1, on=['unique_cust_grp'], how='left').merge(
    feat_2, on=['unique_cust_grp'], how='left').merge(feat_3, on=['unique_cust_grp'], how='left')

In [151]:
print(camp_data_feat.shape)
print(camp_data_feat.columns)
camp_data_feat.head(2)

(794535, 6)
Index(['unique_cust_grp', 'ROW_ID_x', 'repm_vin', 'Bounced_count', 'ROW_ID_y',
       'ROW_ID'],
      dtype='object')


,unique_cust_grp,ROW_ID_x,repm_vin,Bounced_count,ROW_ID_y,ROW_ID
0,0.0,2,1,1.0,NaN,NaN
1,100000.0,1,1,NaN,NaN,1.0


In [152]:
# Merge with target values
camp_data_feat = camp_data_feat.fillna(0)
camp_data_feat['Target_binary'] = np.where(camp_data_feat['unique_cust_grp'].isin(test_0), 0, 1)

In [153]:
camp_data_feat.to_csv('/home/neelima/Service_Latest_Data_Final_Execution/camp_data_feature_ext.dsv', encoding='utf-8', index=False)

In [154]:
camp_data_feat.to_csv('/home/neelima/Service_Latest_Data_Final_Execution/camp_data_feature_ext.csv', encoding='utf-8', index=False)

In [155]:
serv_feat_noused_1.to_csv('/home/neelima/Service_Latest_Data_Final_Execution/ref/Cust_grp_splits/serv_feat_noused_1.dsv', encoding='utf-8', index=False)

In [156]:
serv_rv_feat.to_csv('/home/neelima/Service_Latest_Data_Final_Execution/ref/Cust_grp_splits/repeat_visit/rv_feat_noused_1.dsv', index=False, encoding='utf-8')

In [157]:
serv_grp_1_dlr.to_csv('/home/neelima/Service_Latest_Data_Final_Execution/ref/Cust_grp_splits/dlr_feat/serv_dlr_feat_1.dsv', encoding='utf-8', index=False)

In [158]:
camp_data_feat.head()

,unique_cust_grp,ROW_ID_x,repm_vin,Bounced_count,ROW_ID_y,ROW_ID,Target_binary
0,0.0,2,1,1.0,0.0,0.0,0
1,100000.0,1,1,0.0,0.0,1.0,0
2,1000000.0,2,1,0.0,0.0,1.0,0
3,1000001.0,1,1,1.0,0.0,0.0,0
4,1000002.0,2,1,0.0,0.0,0.0,0


In [160]:
camp_data_feat.to_csv('/home/neelima/Service_Latest_Data_Final_Execution/ref/EDA/Features/camp_data.dsv', encoding='utf-8', index=False)

**Combining split feature files**

In [162]:
import glob
#Combine service features
path = r'/home/neelima/Service_Latest_Data_Final_Execution/ref/Cust_grp_splits' # use your path
all_files = glob.glob(path + "/*.dsv")

li = []

for filename in all_files:
    df = pd.read_csv(filename)
    li.append(df)
rfrl_serv_data = pd.concat(li)

In [163]:
print(rfrl_serv_data.shape)
print(rfrl_serv_data.columns)
rfrl_serv_data.head(3)

(1229214, 96)
Index(['unique_cust_grp', 'FS_mlg_diff mean', 'FS_mlg_diff median',
       'FS_mlg_diff min', 'FS_mlg_diff max', 'PS_mlg_diff mean',
       'PS_mlg_diff median', 'PS_mlg_diff min', 'PS_mlg_diff max',
       'RR_mlg_diff mean', 'RR_mlg_diff median', 'RR_mlg_diff min',
       'RR_mlg_diff max', 'OC_mlg_diff mean', 'OC_mlg_diff median',
       'OC_mlg_diff min', 'OC_mlg_diff max', 'AR_mlg_diff mean',
       'AR_mlg_diff median', 'AR_mlg_diff min', 'AR_mlg_diff max',
       'Milg_diff_vin mean', 'Milg_diff_vin median', 'Milg_diff_vin min',
       'Milg_diff_vin max', 'billable_amount mean', 'billable_amount median',
       'billable_amount min', 'billable_amount max', 'Diff_estimate_bill mean',
       'Diff_estimate_bill median', 'Diff_estimate_bill min',
       'Diff_estimate_bill max', 'FS_day_diff mean', 'FS_day_diff median',
       'FS_day_diff min', 'FS_day_diff max', 'PS_day_diff mean',
       'PS_day_diff median', 'PS_day_diff min', 'PS_day_diff max',
       'RR_day_di

,unique_cust_grp,FS_mlg_diff mean,FS_mlg_diff median,FS_mlg_diff min,FS_mlg_diff max,PS_mlg_diff mean,PS_mlg_diff median,PS_mlg_diff min,PS_mlg_diff max,RR_mlg_diff mean,...,OC_count_vin median,OC_count_vin min,OC_count_vin max,AR_count_vin mean,AR_count_vin median,AR_count_vin min,AR_count_vin max,repm_vin nunique,Purchase_Age mean,service_age_vin mean
0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,15.0,1239.030208
1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,1.0,NaN,NaN,NaN,NaN,1,15.0,0.000000
2,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,1.0,NaN,NaN,NaN,NaN,1,15.0,389.055081


In [164]:
#drop highly correlated features
corr_serv = rfrl_serv_data.corr().abs()
upper = corr_serv.where(np.triu(np.ones(corr_serv.shape), k=1).astype(np.bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.90)]
rfrl_serv_data_red = rfrl_serv_data.drop(rfrl_serv_data[to_drop], axis=1)

In [165]:
print(rfrl_serv_data_red.shape)
print(rfrl_serv_data_red.columns)
rfrl_serv_data_red.head(3)

(1229214, 45)
Index(['unique_cust_grp', 'FS_mlg_diff mean', 'FS_mlg_diff median',
       'FS_mlg_diff min', 'FS_mlg_diff max', 'PS_mlg_diff mean',
       'PS_mlg_diff min', 'PS_mlg_diff max', 'RR_mlg_diff mean',
       'OC_mlg_diff mean', 'OC_mlg_diff min', 'OC_mlg_diff max',
       'AR_mlg_diff mean', 'AR_mlg_diff max', 'Milg_diff_vin mean',
       'Milg_diff_vin min', 'billable_amount mean', 'billable_amount median',
       'billable_amount min', 'billable_amount max', 'Diff_estimate_bill mean',
       'Diff_estimate_bill min', 'FS_day_diff mean', 'FS_day_diff max',
       'PS_day_diff mean', 'PS_day_diff max', 'RR_day_diff mean',
       'RR_day_diff max', 'OC_day_diff mean', 'OC_day_diff max',
       'AR_day_diff mean', 'AR_day_diff max', 'FS_count_vin_norm mean',
       'PS_count_vin_norm mean', 'RR_count_vin_norm mean',
       'OC_count_vin_norm mean', 'AR_count_vin_norm mean', 'FS_count_vin mean',
       'PS_count_vin mean', 'RR_count_vin mean', 'OC_count_vin mean',
       'AR_co

,unique_cust_grp,FS_mlg_diff mean,FS_mlg_diff median,FS_mlg_diff min,FS_mlg_diff max,PS_mlg_diff mean,PS_mlg_diff min,PS_mlg_diff max,RR_mlg_diff mean,OC_mlg_diff mean,...,OC_count_vin_norm mean,AR_count_vin_norm mean,FS_count_vin mean,PS_count_vin mean,RR_count_vin mean,OC_count_vin mean,AR_count_vin mean,repm_vin nunique,Purchase_Age mean,service_age_vin mean
0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,5.0,3.0,NaN,NaN,1,15.0,1239.030208
1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.066667,NaN,NaN,NaN,NaN,1.0,NaN,1,15.0,0.000000
2,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.066667,NaN,NaN,2.0,1.0,1.0,NaN,1,15.0,389.055081


In [166]:
#Combine dealer features
path = r'/home/neelima/Service_Latest_Data_Final_Execution/ref/Cust_grp_splits/dlr_feat' # use your path
all_files = glob.glob(path + "/*.dsv")

li = []

for filename in all_files:
    df = pd.read_csv(filename)
    li.append(df)
dlr_feat = pd.concat(li)

In [167]:
print(dlr_feat.shape)
print(dlr_feat.columns)
dlr_feat.head(3)

(199935, 37)
Index(['unique_cust_grp', 'FS_dlr_count_vin mean', 'FS_dlr_count_vin median',
       'FS_dlr_count_vin min', 'FS_dlr_count_vin max', 'MS_dlr_count_vin mean',
       'MS_dlr_count_vin median', 'MS_dlr_count_vin min',
       'MS_dlr_count_vin max', 'AR_dlr_count_vin mean',
       'AR_dlr_count_vin median', 'AR_dlr_count_vin min',
       'AR_dlr_count_vin max', 'Total_dealer_count nunique',
       'Total_dlr_count_vin mean', 'Total_dlr_count_vin median',
       'Total_dlr_count_vin min', 'Total_dlr_count_vin max',
       'FS_dealer_loyalty_vin mean', 'FS_dealer_loyalty_vin median',
       'FS_dealer_loyalty_vin min', 'FS_dealer_loyalty_vin max',
       'MS_dealer_loyalty_vin mean', 'MS_dealer_loyalty_vin median',
       'MS_dealer_loyalty_vin min', 'MS_dealer_loyalty_vin max',
       'AR_dealer_loyalty_vin mean', 'AR_dealer_loyalty_vin median',
       'AR_dealer_loyalty_vin min', 'AR_dealer_loyalty_vin max',
       'Overall_dealer_loyalty_vin mean', 'Overall_dealer_loyalty_vi

,unique_cust_grp,FS_dlr_count_vin mean,FS_dlr_count_vin median,FS_dlr_count_vin min,FS_dlr_count_vin max,MS_dlr_count_vin mean,MS_dlr_count_vin median,MS_dlr_count_vin min,MS_dlr_count_vin max,AR_dlr_count_vin mean,...,AR_dealer_loyalty_vin median,AR_dealer_loyalty_vin min,AR_dealer_loyalty_vin max,Overall_dealer_loyalty_vin mean,Overall_dealer_loyalty_vin median,Overall_dealer_loyalty_vin min,Overall_dealer_loyalty_vin max,repm_vin nunique,Total_service_count nunique,Max_service_dlr_code
0,0.0,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,NaN,...,NaN,NaN,NaN,8.0,8.0,8.0,8.0,1,8,N1201
1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1,1,E4201
2,2.0,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,NaN,...,NaN,NaN,NaN,2.0,2.0,2.0,2.0,1,4,S4407


In [168]:
#Drop correlated features
corr_serv = dlr_feat.corr().abs()
upper = corr_serv.where(np.triu(np.ones(corr_serv.shape), k=1).astype(np.bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
dlr_feat_red = dlr_feat.drop(dlr_feat[to_drop], axis=1)

In [169]:
print(dlr_feat_red.shape)
print(dlr_feat_red.columns)
dlr_feat_red.head(3)

(199935, 18)
Index(['unique_cust_grp', 'FS_dlr_count_vin mean', 'MS_dlr_count_vin mean',
       'MS_dlr_count_vin min', 'AR_dlr_count_vin mean', 'AR_dlr_count_vin min',
       'Total_dealer_count nunique', 'Total_dlr_count_vin mean',
       'Total_dlr_count_vin min', 'FS_dealer_loyalty_vin mean',
       'MS_dealer_loyalty_vin mean', 'MS_dealer_loyalty_vin min',
       'AR_dealer_loyalty_vin mean', 'AR_dealer_loyalty_vin min',
       'Overall_dealer_loyalty_vin mean', 'Overall_dealer_loyalty_vin min',
       'repm_vin nunique', 'Max_service_dlr_code'],
      dtype='object')


,unique_cust_grp,FS_dlr_count_vin mean,MS_dlr_count_vin mean,MS_dlr_count_vin min,AR_dlr_count_vin mean,AR_dlr_count_vin min,Total_dealer_count nunique,Total_dlr_count_vin mean,Total_dlr_count_vin min,FS_dealer_loyalty_vin mean,MS_dealer_loyalty_vin mean,MS_dealer_loyalty_vin min,AR_dealer_loyalty_vin mean,AR_dealer_loyalty_vin min,Overall_dealer_loyalty_vin mean,Overall_dealer_loyalty_vin min,repm_vin nunique,Max_service_dlr_code
0,0.0,NaN,1.0,1.0,NaN,NaN,1,1.0,1,NaN,8.0,8.0,NaN,NaN,8.0,8.0,1,N1201
1,1.0,NaN,NaN,NaN,NaN,NaN,1,1.0,1,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1,E4201
2,2.0,NaN,1.0,1.0,NaN,NaN,2,2.0,2,NaN,3.0,3.0,NaN,NaN,2.0,2.0,1,S4407


In [170]:
#Combine repeat visit features
path = r'/home/neelima/Service_Latest_Data_Final_Execution/ref/Cust_grp_splits/repeat_visit' # use your path
all_files = glob.glob(path + "/*.dsv")

li = []

for filename in all_files:
    df = pd.read_csv(filename)
    li.append(df)
rv_feat = pd.concat(li)

In [171]:
print(rv_feat.shape)
print(rv_feat.columns)
rv_feat.head(3)

(199936, 12)
Index(['unique_cust_grp', 'rv_3_count', 'serv_count_last_one',
       'serv_count_last_three', 'yearly_rv_3_count mean',
       'yearly_rv_3_count max', 'yearly_rv_3_count median',
       'yearly_rv_3_count min', 'day_diff mean', 'day_diff max',
       'day_diff median', 'day_diff min'],
      dtype='object')


,unique_cust_grp,rv_3_count,serv_count_last_one,serv_count_last_three,yearly_rv_3_count mean,yearly_rv_3_count max,yearly_rv_3_count median,yearly_rv_3_count min,day_diff mean,day_diff max,day_diff median,day_diff min
0,0.0,3.0,2.0,8.0,0.0,0.0,0.0,0.0,176.714286,330.0,135.0,67.0
1,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
2,2.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,129.333333,145.0,144.0,99.0


In [172]:
#Drop correlated features
corr_serv = rv_feat.corr().abs()
upper = corr_serv.where(np.triu(np.ones(corr_serv.shape), k=1).astype(np.bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
rv_feat_red = rv_feat.drop(rv_feat[to_drop], axis=1)

In [173]:
print(rv_feat_red.shape)
print(rv_feat_red.columns)
rv_feat_red.head(3)

(199936, 7)
Index(['unique_cust_grp', 'rv_3_count', 'yearly_rv_3_count mean',
       'yearly_rv_3_count max', 'day_diff mean', 'day_diff max',
       'day_diff min'],
      dtype='object')


,unique_cust_grp,rv_3_count,yearly_rv_3_count mean,yearly_rv_3_count max,day_diff mean,day_diff max,day_diff min
0,0.0,3.0,0.0,0.0,176.714286,330.0,67.0
1,1.0,0.0,0.0,0.0,NaN,NaN,NaN
2,2.0,1.0,0.0,0.0,129.333333,145.0,99.0


In [174]:
rfrl_serv_data_red_new_1 = pd.merge(rfrl_serv_data_red, rv_feat_red, how='inner', on=['unique_cust_grp'])
rfrl_serv_data_red_new_1.shape

(201121, 51)

In [175]:
rfrl_serv_data_red_new_2 = pd.merge(rfrl_serv_data_red_new_1, dlr_feat_red, how='inner', on=['unique_cust_grp'])
rfrl_serv_data_red_new_2.shape

(201121, 68)

In [176]:
camp_data_feat1 = camp_data_feat.copy()
camp_data_feat1['used'] = np.where(((camp_data_feat['unique_cust_grp'].astype(str).str.contains('_used'))), True, False)

In [177]:
print(camp_data_feat1.shape)
camp_data_feat1.used.sum()

(794535, 8)


34348

In [178]:
camp_data_feat1.head()

,unique_cust_grp,ROW_ID_x,repm_vin,Bounced_count,ROW_ID_y,ROW_ID,Target_binary,used
0,0.0,2,1,1.0,0.0,0.0,0,False
1,100000.0,1,1,0.0,0.0,1.0,0,False
2,1000000.0,2,1,0.0,0.0,1.0,0,False
3,1000001.0,1,1,1.0,0.0,0.0,0,False
4,1000002.0,2,1,0.0,0.0,0.0,0,False


In [179]:
camp_data_feat1.columns

Index(['unique_cust_grp', 'ROW_ID_x', 'repm_vin', 'Bounced_count', 'ROW_ID_y',
       'ROW_ID', 'Target_binary', 'used'],
      dtype='object')

In [180]:
camp_data_feat2 = camp_data_feat1[camp_data_feat1['used']==False][['unique_cust_grp', 'ROW_ID_x', 'repm_vin', 'ROW_ID_y', 'ROW_ID_x',
       'ROW_ID_y', 'Target_binary', 'used']]
print(camp_data_feat2.shape)
print(camp_data_feat2.columns)
camp_data_feat2.head()

(760187, 8)
Index(['unique_cust_grp', 'ROW_ID_x', 'repm_vin', 'ROW_ID_y', 'ROW_ID_x',
       'ROW_ID_y', 'Target_binary', 'used'],
      dtype='object')


,unique_cust_grp,ROW_ID_x,repm_vin,ROW_ID_y,ROW_ID_x,ROW_ID_y,Target_binary,used
0,0.0,2,1,0.0,2,0.0,0,False
1,100000.0,1,1,0.0,1,0.0,0,False
2,1000000.0,2,1,0.0,2,0.0,0,False
3,1000001.0,1,1,0.0,1,0.0,0,False
4,1000002.0,2,1,0.0,2,0.0,0,False


In [181]:
camp_data_feat2["unique_cust_grp"] = camp_data_feat2["unique_cust_grp"].astype(float)

In [182]:
camp_data_feat2 = camp_data_feat2[['unique_cust_grp', 'ROW_ID_x', 'ROW_ID_x', 'repm_vin', 'ROW_ID_y',
       'ROW_ID_y', 'ROW_ID_x', 'ROW_ID_x', 'ROW_ID_y', 'ROW_ID_y',
       'Target_binary']]

In [183]:
rfrl_serv_data_red_new_4 = pd.merge(rfrl_serv_data_red_new_2, camp_data_feat2, how='left', on=['unique_cust_grp'])
rfrl_serv_data_red_new_4.shape

(201121, 86)

In [185]:
pd.set_option('display.max_columns', 200)

In [186]:
print(rfrl_serv_data_red_new_4.shape)
display(rfrl_serv_data_red_new_4.columns)
rfrl_serv_data_red_new_4.head()

(201121, 86)


Index(['unique_cust_grp', 'FS_mlg_diff mean', 'FS_mlg_diff median',
       'FS_mlg_diff min', 'FS_mlg_diff max', 'PS_mlg_diff mean',
       'PS_mlg_diff min', 'PS_mlg_diff max', 'RR_mlg_diff mean',
       'OC_mlg_diff mean', 'OC_mlg_diff min', 'OC_mlg_diff max',
       'AR_mlg_diff mean', 'AR_mlg_diff max', 'Milg_diff_vin mean',
       'Milg_diff_vin min', 'billable_amount mean', 'billable_amount median',
       'billable_amount min', 'billable_amount max', 'Diff_estimate_bill mean',
       'Diff_estimate_bill min', 'FS_day_diff mean', 'FS_day_diff max',
       'PS_day_diff mean', 'PS_day_diff max', 'RR_day_diff mean',
       'RR_day_diff max', 'OC_day_diff mean', 'OC_day_diff max',
       'AR_day_diff mean', 'AR_day_diff max', 'FS_count_vin_norm mean',
       'PS_count_vin_norm mean', 'RR_count_vin_norm mean',
       'OC_count_vin_norm mean', 'AR_count_vin_norm mean', 'FS_count_vin mean',
       'PS_count_vin mean', 'RR_count_vin mean', 'OC_count_vin mean',
       'AR_count_vin mean',

,unique_cust_grp,FS_mlg_diff mean,FS_mlg_diff median,FS_mlg_diff min,FS_mlg_diff max,PS_mlg_diff mean,PS_mlg_diff min,PS_mlg_diff max,RR_mlg_diff mean,OC_mlg_diff mean,OC_mlg_diff min,OC_mlg_diff max,AR_mlg_diff mean,AR_mlg_diff max,Milg_diff_vin mean,Milg_diff_vin min,billable_amount mean,billable_amount median,billable_amount min,billable_amount max,Diff_estimate_bill mean,Diff_estimate_bill min,FS_day_diff mean,FS_day_diff max,PS_day_diff mean,PS_day_diff max,RR_day_diff mean,RR_day_diff max,OC_day_diff mean,OC_day_diff max,AR_day_diff mean,AR_day_diff max,FS_count_vin_norm mean,PS_count_vin_norm mean,RR_count_vin_norm mean,OC_count_vin_norm mean,AR_count_vin_norm mean,FS_count_vin mean,PS_count_vin mean,RR_count_vin mean,OC_count_vin mean,AR_count_vin mean,repm_vin nunique_x,Purchase_Age mean,service_age_vin mean,rv_3_count,yearly_rv_3_count mean,yearly_rv_3_count max,day_diff mean,day_diff max,day_diff min,FS_dlr_count_vin mean,MS_dlr_count_vin mean,MS_dlr_count_vin min,AR_dlr_count_vin mean,AR_dlr_count_vin min,Total_dealer_count nunique,Total_dlr_count_vin mean,Total_dlr_count_vin min,FS_dealer_loyalty_vin mean,MS_dealer_loyalty_vin mean,MS_dealer_loyalty_vin min,AR_dealer_loyalty_vin mean,AR_dealer_loyalty_vin min,Overall_dealer_loyalty_vin mean,Overall_dealer_loyalty_vin min,repm_vin nunique_y,Max_service_dlr_code,ROW_ID_x,ROW_ID_x,ROW_ID_x,ROW_ID_x,repm_vin,ROW_ID_y,ROW_ID_y,ROW_ID_y,ROW_ID_y,ROW_ID_x,ROW_ID_x,ROW_ID_x,ROW_ID_x,ROW_ID_y,ROW_ID_y,ROW_ID_y,ROW_ID_y,Target_binary
0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6096.640000,5281.525,0.0,13884.85,NaN,NaN,NaN,NaN,309.757552,330.018657,149.913819,202.807639,NaN,NaN,NaN,NaN,NaN,0.333333,0.200000,NaN,NaN,NaN,5.0,3.0,NaN,NaN,1,15.0,1239.030208,3.0,0.0,0.0,176.714286,330.0,67.0,NaN,1.0,1.0,NaN,NaN,1,1.0,1,NaN,8.0,8.0,NaN,NaN,8.0,8.0,1,N1201,2.0,2.0,2.0,2.0,1.0,0.0,0.0,0.0,0.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0
1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6096.640000,5281.525,0.0,13884.85,NaN,NaN,NaN,NaN,309.757552,330.018657,149.913819,202.807639,NaN,NaN,NaN,NaN,NaN,0.333333,0.200000,NaN,NaN,NaN,5.0,3.0,NaN,NaN,1,15.0,1239.030208,0.0,0.0,0.0,176.714286,330.0,67.0,NaN,1.0,1.0,NaN,NaN,1,1.0,1,NaN,8.0,8.0,NaN,NaN,8.0,8.0,1,N1201,2.0,2.0,2.0,2.0,1.0,0.0,0.0,0.0,0.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0
2,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000,0.0,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.066667,NaN,NaN,NaN,NaN,1.0,NaN,1,15.0,0.000000,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1.0,1,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1,E4201,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3306.752500,2762.710,0.0,7701.59,NaN,NaN,NaN,NaN,245.050914,245.050914,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.133333,0.066667,0.066667,NaN,NaN,2.0,1.0,1.0,NaN,1,15.0,389.055081,1.0,0.0,0.0,129.333333,145.0,99.0,NaN,1.0,1.0,NaN,NaN,2,2.0,2,NaN,3.0,3.0,NaN,NaN,2.0,2.0,1,S4407,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,10867.0,10867.0,5500.471667,5538.420,0.0,15142.68,-5500.471667,-15142.68,NaN,NaN,262.393511,476.347153,249.649132,472.927928,NaN,NaN,NaN,NaN,NaN,0.466667,0.266667,0.066667,NaN,NaN,7.0,4.0,1.0,NaN,1,15.0,1574.361065,5.0,1.0,1.0,142.636364,391.0,23.0,NaN,2.0,2.0,NaN,NaN,2,2.0,2,NaN,5.5,5.5,NaN,NaN,6.0,6.0,1,S8604,2.0,2.0,2.0,2.0,1.0,0.0,0.0,0.0,0.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0


In [188]:
rfrl_serv_data_red_new_4.to_csv('/home/neelima/Service_Latest_Data_Final_Execution/ref/EDA/Features/rfrl_serv_data_red_new.dsv', encoding='utf-8', index=False)

In [187]:
cust_grp.head()

,con_id,con_id_x,con_id_y,phone_no,repm_vin,ser_ph_grp,service_phone,unique_cust_grp,vin_grp,Target,used_1,used_2,used_vin
0,NaN,1-29OZ-1503,1-29OZ-1503,9.89111e+09,MALAA51HR5M700082,1316486,9891112121,0,566852,0,False,False,False
1,NaN,1-29OE-2971,1-29OE-2971,2.5012e+09,MALAA51HR5M700158,235,2501201900,1,566853,0,False,False,False
2,NaN,1-2A9X-3345,1-2A9X-3345,9.99416e+09,MALAA51HR5M700178,1567815,9994155040,2,566854,0,False,False,False
3,NaN,1-2B7U-3041,1-2B7U-3041,9.88098e+09,MALAA51HR5M700666,672422,9448346265,3,566855,0,False,False,False
4,NaN,1-2B7U-3041,1-F817-67,9.88098e+09,MALAA51HR5M700666,1284729,9880977808,3,566855,0,False,False,False


In [190]:
refrl_ser_rcrepm_tb.head(3)

,repm_dlr_no,repm_ro_no,repm_vin,repm_cust_no,repm_ro_stat,repm_estmt_no,repm_work_type,repm_milg,repm_estmt_amt,repm_ro_dtime,repm_bill_total_part_amt,repm_bill_total_labr_amt,repm_bill_total_othr_amt,repm_total_labr_amt,repm_total_part_amt,repm_total_othr_amt,repm_vst_type,repm_vst_type_seq_no,repm_rmkrs,repm_crte_dtime,repm_prvs_milg,repm_used_flag,repm_byng_dlr_code,repm_no_details,repm_ro_stat_new,repm_work_category,repm_dlr_code,repm_exit_milg,repm_file_doc_id,repm_ro_mobile_flag,model
0,S5819,R202000370,MALC841DMHM027739,C2019113419,D,NaN,PS,27252,0.00,2020-06-04,6316.96,4245.64,0.00,0.00,0.00,0.00,NaN,0.0,NaN,2020-06-04,18186,N,NaN,NaN,UR,NaN,S5819,27253.0,NaN,NaN,Next Gen Verna
1,N8803,R202000558,MALAF51CLLM109400,C2020023561,D,NaN,AR,1080,0.00,2020-06-09,822.00,11210.00,0.00,0.00,0.00,0.00,NaN,0.0,NaN,2020-06-09,1050,N,NaN,NaN,UR,AC,N8803,1085.0,NaN,NaN,Santro
2,S5227,R202003538,MALAA51HR8M278240,C2016139404,D,NaN,RR,50000,0.00,2020-06-24,357.99,0.00,0.00,0.00,0.00,0.00,NaN,0.0,NaN,2020-06-24,48980,N,NaN,NaN,AD,NaN,S5227,50000.0,NaN,NaN,Santro


In [189]:
cust_grp.to_csv("used_vin.csv")

In [191]:
cust_grp = pd.merge(cust_grp, refrl_ser_rcrepm_tb[['repm_vin','model','repm_dlr_no','repm_work_type']], how ='left', on ='repm_vin')

In [192]:
cust_grp.head(5)

,con_id,con_id_x,con_id_y,phone_no,repm_vin,ser_ph_grp,service_phone,unique_cust_grp,vin_grp,Target,used_1,used_2,used_vin,model,repm_dlr_no,repm_work_type
0,NaN,1-29OZ-1503,1-29OZ-1503,9.89111e+09,MALAA51HR5M700082,1316486,9891112121,0,566852,0,False,False,False,NaN,NaN,NaN
1,NaN,1-29OE-2971,1-29OE-2971,2.5012e+09,MALAA51HR5M700158,235,2501201900,1,566853,0,False,False,False,NaN,NaN,NaN
2,NaN,1-2A9X-3345,1-2A9X-3345,9.99416e+09,MALAA51HR5M700178,1567815,9994155040,2,566854,0,False,False,False,NaN,NaN,NaN
3,NaN,1-2B7U-3041,1-2B7U-3041,9.88098e+09,MALAA51HR5M700666,672422,9448346265,3,566855,0,False,False,False,Santro,S8205,PS
4,NaN,1-2B7U-3041,1-F817-67,9.88098e+09,MALAA51HR5M700666,1284729,9880977808,3,566855,0,False,False,False,Santro,S8205,PS


In [193]:
cust_grp['model'].value_counts()

New i20              280432
Grand i10            275952
Creta                237257
Eon                  103086
i10                   90299
Xcent                 79558
Next Gen Verna        62477
Santro                59807
New Verna             56417
i20                   55056
i20 Active            44120
Tucson                 4951
Neo Elantra            4712
Verna                  4599
Elantra                4300
Venue                  4016
New Santa Fe (DM)      3181
Accent                 2839
Getz                   2030
Grand i10 NIOS         1114
Santa Fe                659
Santa Fe CBU            158
New Sonata              155
Sonata                  120
Kona EV                  74
E.F.Sonata               16
Terracan                  8
New Creta                 6
AURA                      2
Name: model, dtype: int64

In [196]:
cust_grp['Referrals_count'] = cust_grp.groupby(['repm_vin'])['unique_cust_grp'].transform('count')

In [197]:
cust_grp.head(10)

,con_id,con_id_x,con_id_y,phone_no,repm_vin,ser_ph_grp,service_phone,unique_cust_grp,vin_grp,Target,used_1,used_2,used_vin,model,repm_dlr_no,repm_work_type,Referrals_count
0,NaN,1-29OZ-1503,1-29OZ-1503,9.89111e+09,MALAA51HR5M700082,1316486,9891112121,0,566852,0,False,False,False,NaN,NaN,NaN,1
1,NaN,1-29OE-2971,1-29OE-2971,2.5012e+09,MALAA51HR5M700158,235,2501201900,1,566853,0,False,False,False,NaN,NaN,NaN,1
2,NaN,1-2A9X-3345,1-2A9X-3345,9.99416e+09,MALAA51HR5M700178,1567815,9994155040,2,566854,0,False,False,False,NaN,NaN,NaN,1
3,NaN,1-2B7U-3041,1-2B7U-3041,9.88098e+09,MALAA51HR5M700666,672422,9448346265,3,566855,0,False,False,False,Santro,S8205,PS,2
4,NaN,1-2B7U-3041,1-F817-67,9.88098e+09,MALAA51HR5M700666,1284729,9880977808,3,566855,0,False,False,False,Santro,S8205,PS,2
5,NaN,1-2B0Z-4957,1-2B0Z-4957,9.41558e+09,MALAA51HR5M700709,529983,9415575697,4,566856,0,False,False,False,NaN,NaN,NaN,2
6,NaN,1-2B0Z-4957,1-2AVC-1403,9.41558e+09,MALAA51HR5M700709,1163016,9839904042,4,566856,0,False,False,False,NaN,NaN,NaN,2
7,NaN,1-2A2R-1128,1-2A2R-1128,9.6862e+09,MALAA51HR5M700749,870228,9686203532,5,566857,0,False,False,False,NaN,NaN,NaN,1
8,NaN,1-2A5L-3945,1-JAQZ-574,9.34487e+09,MALAA51HR5M700751,474737,9344873535,6,566858,0,False,False,False,Santro,S4230,PS,1
9,NaN,1-2BEL-2312,1-2BEL-2312,9.32229e+09,MALAA51HR5M700870,462158,9322291031,7,566859,0,False,False,False,NaN,NaN,NaN,1


In [230]:
cust_grp = pd.merge(cust_grp,test_date[['repm_vin','Purchase_age','Age_of_car']], how ='left', on ='repm_vin')

In [234]:
cust_grp.head(4)

,con_id,con_id_x,con_id_y,phone_no,repm_vin,ser_ph_grp,service_phone,unique_cust_grp,vin_grp,Target,used_1,used_2,used_vin,model,repm_dlr_no,repm_work_type,Referrals_count,Purchase_age,Age_of_car
0,NaN,1-29OZ-1503,1-29OZ-1503,9.89111e+09,MALAA51HR5M700082,1316486,9891112121,0,566852,0,False,False,False,NaN,NaN,NaN,1,15.0,15.0
1,NaN,1-29OZ-1503,1-29OZ-1503,9.89111e+09,MALAA51HR5M700082,1316486,9891112121,0,566852,0,False,False,False,NaN,NaN,NaN,1,15.0,15.0
2,NaN,1-29OZ-1503,1-29OZ-1503,9.89111e+09,MALAA51HR5M700082,1316486,9891112121,0,566852,0,False,False,False,NaN,NaN,NaN,1,15.0,15.0
3,NaN,1-29OZ-1503,1-29OZ-1503,9.89111e+09,MALAA51HR5M700082,1316486,9891112121,0,566852,0,False,False,False,NaN,NaN,NaN,1,15.0,15.0


In [235]:
cust_grp.to_csv("used_vin_final_latest.csv")